In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
from sympy import *
extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

In [31]:
#  15112020 reconfinement
JE,JI=2.0,1.5
a,b=0.6,-0.9#0.3,0.2#
# calculate the rotation matrix in rank-2 case (reduced), then extend to large 
# generate perturbed matrix
J2=np.zeros((2,2))
J2[:,0],J2[:,1]=JE,-JI
J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
lvec2,svr2,rvech2=la.svd(J2)
M2 = np.array([[1,1],[1,-1]])
ROT = np.sqrt(2)*la.inv(M2)@lvec2 # M@ROT.T/sqrt(2?)=u
print('Rotation:',ROT)
ROT_=ROT.copy()
ROT_[0,:]=ROT[1,:]
ROT_[1,:]=ROT[0,:]
ROT_[:,0]*=(-1)


'''
Generate two expressions in large random matrix
params:
    nn = E/I neuron number
    M/Nrank = low rank structure in EI format
    U/V.    = low rank structyre in SVD format
    M/Nranktilde = low rank EI format for large random matrix
    U/Vtilde = low rank SVD format for large random matrix
    sigvmat: diagonal singular values matrix
    

'''
grandom=0.2
theta = grandom
Rc,R = 0,2
for i in range(2):
    if svr2[i]>grandom:
        Rc+=1
nn=600
sigvmat,rhosigv = np.zeros((R,R)),np.zeros((R,R))
for i in range(R):
    sigvmat[i,i]=svr2[i]
    rhosigv[i,i]=svr2[i]+(grandom**2)/svr2[i]
invsigvmat = la.inv(sigvmat)
# print('inverse singular value matrix:',invsigvmat)
# print('square of the inverse singular value matrix:',la.matrix_power(invsigvmat,2)) 

zetau,zetav = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
# zetau= np.random.randn(nn*2,R)/np.sqrt(2*nn)
# zetau[:,1]=zetau[:,0]#np.sqrt(0.8)*zetau[:,0]+np.sqrt(0.2)*np.squeeze(np.random.randn(nn*2,1))/np.sqrt(2*nn)
zetav = zetau # correlation between random terms
zetaM,zetaN = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
zetaM_,zetaN_ = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
# generate rank-2 matrix P without random entries
P=np.zeros((nn*2,nn*2))
P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
# use SVD to decompose the low rank matrix
lvec,svr,rvech=la.svd(P)
rvec=rvech.T 
# check the singular value
# print(svr,svr2)
if np.sum((svr[:2]-svr2[:])**2)>1e-8:  
    print('Singular values for 2by2 matrix do not match those for NbyN matrix!')

alphamat= np.eye(R)-(grandom**2)*la.matrix_power(invsigvmat,2)
Asqrt=np.eye(R)#alphamat.copy()
invAsqrt = np.zeros_like(Asqrt)
for i in range(R):
    if alphamat[i,i]>0.0:
        Asqrt[i,i]=np.sqrt(alphamat[i,i])
        invAsqrt[i,i]=1.0/Asqrt[i,i]
    else:
        Asqrt[i,i]=1e-3
        invAsqrt[i,i]=1.0/Asqrt[i,i]
randomterm=np.zeros((R,R))
for i in range(R):
    randomterm[i,i]=np.sqrt(1.0-Asqrt[i,i]**2)
# EI format
withrand,wrank = 1,1
M1,N1=np.ones(2*nn),np.ones(2*nn)
N1[:nn],N1[nn:]=JE/nn,-JI/nn
M2,N2=M1.copy(),N1.copy()
M2[:nn],M2[nn:]=1,-1
N2[:nn],N2[nn:]=a/nn,b/nn
Mrank,Nrank=np.zeros((nn*2,R)),np.zeros((nn*2,R))
NT = np.random.randn(nn*2,R)
NT = NT.T 
if R>1:
    Mrank[:,0],Mrank[:,1],Nrank[:,0],Nrank[:,1]=M1,M2,N1,N2
else:
    Mrank[:,0],Nrank[:,0]=M1,N1
zetaMtilde = np.sqrt(2*nn)*zetaM*grandom@invsigvmat@la.inv(Asqrt)@ROT.T
zetaNtilde = 1.0/np.sqrt(2*nn)*zetaN*grandom@la.inv(Asqrt)@ROT.T
'''orthogonal'''
c=1.0
zetaMtilde_ = np.sqrt(2*nn)*zetaM_*grandom@invsigvmat@la.inv(Asqrt)@ROT.T#_#
zetaNtilde_ = 1.0/np.sqrt(2*nn)*zetaN_*grandom@la.inv(Asqrt)@ROT.T#_#
# zetaMtilde_ = np.sqrt(2*nn)*zetaM_*grandom@invsigvmat@invAsqrt@ROT_#
# zetaNtilde_ = 1.0/np.sqrt(2*nn)*zetaN_*grandom@invAsqrt@ROT_#
Mranktilde=(Mrank+zetaMtilde_*c+zetaMtilde*(1-c))@ROT@Asqrt
Nranktilde=(Nrank+zetaNtilde_*c+zetaNtilde*(1-c))@ROT@Asqrt@invsigvmat@(sigvmat+grandom**2*invsigvmat)
Nranktildeh = Nranktilde.T
U,V=np.zeros_like(Mrank),np.zeros_like(Nrank)
U,V=lvec[:,:R],rvec[:,:R]
Utilde,Vtilde = wrank*U@Asqrt+withrand*zetau@randomterm,wrank*V@Asqrt+withrand*zetav@randomterm

# add random part in, Xtilde = X+P
X = (grandom)*np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
Xtilde=X+P
Utdnum,sigvtdnum,Vtdnumh=la.svd(Xtilde)
Utdnum2,sigvtdnum2,Vtdnumh2=Utdnum[:,:Rc],sigvtdnum[:Rc],Vtdnumh[:Rc,:]
Vtdnum2=Vtdnumh2.T
for i in range(Rc):
    Vtdnum2[:,i]*=sigvtdnum2[i]
Xtilde_num=Vtdnum2.T@Utdnum2
if Rc>1:
    Xtilde_svdrec = (sigvmat+(grandom**2)*invsigvmat)@Vtilde.T@Utilde#Utilde@(sigvmat+(grandom**2)*invsigvmat)@Vtilde.T#/grandom**2
    Xtilde_eirec  = Nranktildeh@Mranktilde#Nranktildeh@Mranktilde#/grandom**2
    Xtilde_low = Nrank.T@Mrank
else:
    Xtilde_svdrec = np.reshape(Vtilde[:,0],(1,2*nn))*(sigvmat[0,0]+(grandom**2)*invsigvmat[0,0])@np.reshape(Utilde[:,0],(nn*2,1))#@#/grandom**2
    Xtilde_eirec  = np.reshape(Nranktilde[:,0],(1,2*nn))@np.reshape(Mranktilde[:,0],(2*nn,1))#np.reshape(Nranktildeh[0,:],(1,2*nn))@np.reshape(Mranktilde[:,0],(nn*2,1))#/grandom**2
    Xtilde_low = np.reshape(Nrank[:,0],(1,nn*2))@np.reshape(Mrank[:,0],(nn*2,1))

# if Rc>1:
#     Xtilde_svdrec = Utilde@(sigvmat+(grandom**2)*invsigvmat)@Vtilde.T#/grandom**2
#     Xtilde_eirec  = Mranktilde@Nranktildeh#Nranktildeh@Mranktilde#/grandom**2
#     Xtilde_low = Mrank@Nrank.T
# else:
#     Xtilde_svdrec = np.reshape(Utilde[:,0],(2*nn,1))*(sigvmat[0,0]+(grandom**2)*invsigvmat[0,0])@np.reshape(Vtilde[:,0],(1,nn*2))#@#/grandom**2
#     Xtilde_eirec  = np.reshape(Mranktilde[:,0],(2*nn,1))@np.reshape(Nranktilde[:,0],(1,2*nn))
#     Xtilde_low = np.reshape(Mrank[:,0],(nn*2,1))@np.reshape(Nrank[:,0],(1,nn*2))

# test two expressions, using singular values
_,sigvxtilde,_=la.svd(Xtilde_num)#Xtilde)
_,sigveitilde,_=la.svd(Xtilde_eirec)
_,sigvsvdtilde,_=la.svd(Xtilde_svdrec)

_,sigvlow,_=la.svd(Xtilde_low)

print('compare the singular values from')
print('original:',sigvxtilde[:Rc])
print('svd:',sigvsvdtilde[:Rc])
print('ei:',sigveitilde[:Rc])#Xtilde_eirec)#
print('low:',sigvlow[:Rc])




Rotation: [[-0.92350317  0.38359079]
 [-0.38359079 -0.92350317]]
compare the singular values from
original: [3.54652139 0.4901756 ]
svd: [3.5189723  0.53913121]
ei: [3.50171094 0.49694515]
low: [3.82339652 0.47078559]


In [32]:
print(U[:2,0],U[nn:nn+2,0],U[:2,1],U[nn:nn+2,1])

[-0.03773255 -0.03773255] [-0.01558593 -0.01558593] [-0.01558593 -0.01558593] [0.03773255 0.03773255]


In [95]:
ROT_=ROT.copy()
ROT_[0,:]=ROT[1,:]
ROT_[1,:]=ROT[0,:]
ROT_[:,0]*=(-1)
print(ROT_@ROT)
# Mranktilde,Mrank,Nranktilde,Nrank=-Mranktilde/np.sqrt(2*nn),-Mrank/np.sqrt(2*nn),-Nranktilde*np.sqrt(2*nn),-Nrank*np.sqrt(2*nn)
# zetaMtilde,zetaNtilde=-zetaMtilde/np.sqrt(2*nn),-zetaNtilde*np.sqrt(2*nn)
zetaMtilde_ = np.sqrt(2*nn)*zetaM*grandom@invsigvmat@la.inv(Asqrt)@ROT_
zetaNtilde_ = 1.0/np.sqrt(2*nn)*zetaN*grandom@la.inv(Asqrt)@ROT_
zetaMtilde_,zetaNtilde_=-zetaMtilde_/np.sqrt(2*nn),-zetaNtilde_*np.sqrt(2*nn)
fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(111)
ax1.scatter(-(Mrank[:,0]+zetaMtilde[:,0]),-(Mrank[:,1]+zetaMtilde[:,1]),c='pink',alpha=0.5)
ax1.scatter(-(Mrank[:,0]+zetaMtilde_[:,0]),-(Mrank[:,1]+zetaMtilde_[:,1]),c='green',alpha=0.5)
ax1.set_aspect('equal')

ax1.set_xlabel(r'$\tilde{M}^{(1)}$',fontsize=14)
ax1.set_ylabel(r'$\tilde{M}^{(2)}$',fontsize=14)
ax1.set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+', $a, b=$'+str(a)+','+str(b)+'\n $\sigma_1=$'+str(format(svr2[0],'.1f'))+r',$\rho_1=$'+str(format(rhosigv[0,0],'.1f'))+',$\overline{\sigma}=$'+str(grandom),fontsize=16)

[[-5.55111512e-17  1.00000000e+00]
 [-1.00000000e+00  5.55111512e-17]]


Text(0.5, 1.0, '$J_E,\\ J_I=$2.9,1.5, $a, b=$0.6,-0.9\n $\\sigma_1=$4.8,$\\rho_1=$4.9,$\\overline{\\sigma}=$0.5')

In [48]:
# SCATTER PLOT of Mtilde and Ntilde
bimeanMtilde,bimeanNtilde=np.zeros((R,Rc)),np.zeros((R,Rc))
# Mranktilde,Mrank,Nranktilde,Nrank=-Mranktilde/np.sqrt(2*nn),-Mrank/np.sqrt(2*nn),-Nranktilde*np.sqrt(2*nn),-Nrank*np.sqrt(2*nn)
# zetaMtilde,zetaNtilde=-zetaMtilde/np.sqrt(2*nn),-zetaNtilde*np.sqrt(2*nn)

for i in range(Rc):
    bimeanMtilde[0,i],bimeanNtilde[0,i]=np.mean(Mranktilde[:nn,i]),np.mean(Nranktilde[:nn,i])
    bimeanMtilde[1,i],bimeanNtilde[1,i]=np.mean(Mranktilde[nn:,i]),np.mean(Nranktilde[nn:,i])
RANKP=0
fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(121)
ax1.scatter(-(Mrank[:,0]+zetaMtilde[:,0]),-(Mrank[:,1]+zetaMtilde[:,1]),c='pink',alpha=0.5)
ax1.set_aspect('equal')
ax2=fig.add_subplot(122)
ax2.scatter(-(Nrank[:,0]+zetaNtilde[:,0]),-(Nrank[:,1]+zetaNtilde[:,1]),c='green',alpha=0.5)
ax2.set_aspect('equal')

fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(121)
ax1.scatter(-(Mrank[:,0]+zetaMtilde[:,0]),-(Nrank[:,0]+zetaNtilde[:,0]),c='pink',alpha=0.5)
ax1.set_aspect('equal')
ax2=fig.add_subplot(122)
ax2.scatter(-(Mrank[:,1]+zetaMtilde[:,1]),-(Nrank[:,1]+zetaNtilde[:,1]),c='green',alpha=0.5)
ax2.set_aspect('equal')

fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(121)
ax1.scatter(-(Mrank[:,0]+zetaMtilde[:,0]),-(Nrank[:,1]+zetaNtilde[:,1]),c='pink',alpha=0.5)
ax1.set_aspect('equal')
ax2=fig.add_subplot(122)
ax2.scatter(-(Mrank[:,1]+zetaMtilde[:,1]),-(Nrank[:,0]+zetaNtilde[:,0]),c='green',alpha=0.5)
ax2.set_aspect('equal')


In [34]:
# SCATTER PLOT of Mtilde and Ntilde
bimeanMtilde,bimeanNtilde=np.zeros((R,Rc)),np.zeros((R,Rc))
# Mranktilde,Mrank,Nranktilde,Nrank=-Mranktilde/np.sqrt(2*nn),-Mrank/np.sqrt(2*nn),-Nranktilde*np.sqrt(2*nn),-Nrank*np.sqrt(2*nn)
# zetaMtilde,zetaNtilde=-zetaMtilde/np.sqrt(2*nn),-zetaNtilde*np.sqrt(2*nn)

for i in range(Rc):
    bimeanMtilde[0,i],bimeanNtilde[0,i]=np.mean(Mranktilde[:nn,i]),np.mean(Nranktilde[:nn,i])
    bimeanMtilde[1,i],bimeanNtilde[1,i]=np.mean(Mranktilde[nn:,i]),np.mean(Nranktilde[nn:,i])
RANKP=0
fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(111)
ax1.scatter(-(Mrank[:,RANKP]+zetaMtilde[:,RANKP]),-(Nrank[:,RANKP]+zetaNtilde[:,RANKP]),c='pink',alpha=0.5)
ax1.set_aspect('equal')
ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)
RANKP=1
fig=plt.figure(figsize=(6,6))
ax2=fig.add_subplot(111)
ax2.scatter(-(Mrank[:,RANKP]+zetaMtilde[:,RANKP]),-(Nrank[:,RANKP]+zetaNtilde[:,RANKP]),c='pink',alpha=0.5)
ax2.set_aspect('equal')
ax2.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax2.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

nVtilde = Vtilde@(sigvmat+(grandom**2)*invsigvmat)
nV = V@sigvmat
bimeanUtilde,bimeanVtilde=np.zeros((R,R)),np.zeros((R,R))
for i in range(Rc):
    bimeanUtilde[0,i],bimeanVtilde[0,i]=np.mean(Utilde[:nn,i]),np.mean(nVtilde[:nn,i])
    bimeanUtilde[1,i],bimeanVtilde[1,i]=np.mean(Utilde[nn:,i]),np.mean(nVtilde[nn:,i])
RANKP=0
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax1.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax1.set_aspect('equal')
ax1.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax1.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

RANKP=1
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax2.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax2.set_aspect('equal')
ax2.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax2.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

ax2.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax2.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)


In [344]:
#  15112020 reconfinement
JE,JI=2.9,1.5
a,b=0.6,-0.9#0.3,0.2#
# calculate the rotation matrix in rank-2 case (reduced), then extend to large 
# generate perturbed matrix
J2=np.zeros((2,2))
J2[:,0],J2[:,1]=JE,-JI
J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
lvec2,svr2,rvech2=la.svd(J2)
M2 = np.array([[1,1],[1,-1]])
ROT = np.sqrt(2)*la.inv(M2)@lvec2 # M@ROT.T/sqrt(2?)=u
print('Rotation:',ROT)

'''
Generate two expressions in large random matrix
params:
    nn = E/I neuron number
    M/Nrank = low rank structure in EI format
    U/V.    = low rank structyre in SVD format
    M/Nranktilde = low rank EI format for large random matrix
    U/Vtilde = low rank SVD format for large random matrix
    sigvmat: diagonal singular values matrix
    

'''
grandom=0.5
theta = grandom
Rc,R = 0,2
for i in range(2):
    if svr2[i]>grandom:
        Rc+=1
nn=600
sigvmat,rhosigv = np.zeros((R,R)),np.zeros((R,R))
for i in range(R):
    sigvmat[i,i]=svr2[i]
invsigvmat = la.inv(sigvmat)

zetau,zetav = grandom*np.random.randn(nn*2,R)/np.sqrt(2*nn),grandom*np.random.randn(nn*2,R)/np.sqrt(2*nn)
zetaM,zetaN = grandom*np.random.randn(nn*2,R)/np.sqrt(2*nn),grandom*np.random.randn(nn*2,R)/np.sqrt(2*nn)
# generate rank-2 matrix P without random entries
P=np.zeros((nn*2,nn*2))
P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
# use SVD to decompose the low rank matrix
lvec,svr,rvech=la.svd(P)
rvec=rvech.T 
# check the singular value
# print(svr,svr2)
if np.sum((svr[:2]-svr2[:])**2)>1e-8:  
    print('Singular values for 2by2 matrix do not match those for NbyN matrix!')

alphamat= np.eye(R)-(grandom**2)*la.matrix_power(invsigvmat,2)
Asqrt=np.eye(R)#alphamat.copy()
for i in range(R):
    if alphamat[i,i]>0.0:
        Asqrt[i,i]=np.sqrt(alphamat[i,i])
    else:
        Asqrt[i,i]=0.0
randomterm=np.zeros((R,R))
for i in range(R):
    randomterm[i,i]=np.sqrt(1.0-Asqrt[i,i]**2)

U,V=np.zeros_like(Mrank),np.zeros_like(Nrank)
U,V=lvec[:,:R],rvec[:,:R]

# add random part in, Xtilde = X+P
X = (grandom)*np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
Xtilde=X+P
# test two expressions, using singular values
lvectd,sigvxtilde,rvechtd=la.svd(Xtilde)
Utilde,Vtilde = lvectd[:,:R],rvechtd[:R,:]
Vtilde = Vtilde.T



Rotation: [[-0.95863948  0.28462317]
 [-0.28462317 -0.95863948]]


In [345]:
print('original sigv:',svr[:2],'; critical value:',grandom)
print('random sigv num:',sigvxtilde[:2],';')
theosigv = svr[:2]+grandom**2/svr[:2]
print('theoretical sigv:',theosigv)


original sigv: [4.81194992 0.71073059] ; critical value: 0.5
random sigv num: [4.86061557 1.0633878 ] ;
theoretical sigv: [4.86390391 1.06248131]


In [346]:
print('overlap bewteen singular vectors, num:',np.sum(U[:,0]*Utilde[:,0])**2,';right ',np.sum(V[:,0]*Vtilde[:,0])**2)
print('theoretical:',Asqrt[0,0]**2)
print('overlap bewteen singular vectors, num:',np.sum(U[:,1]*Utilde[:,1])**2,';right ',np.sum(V[:,1]*Vtilde[:,1])**2)
print('theoretical:',Asqrt[1,1]**2)

overlap bewteen singular vectors, num: 0.9888076931786727 ;right  0.9888560118383901
theoretical: 0.9892031315204817
overlap bewteen singular vectors, num: 0.5356105817001876 ;right  0.5493838003262217
theoretical: 0.5050857094904272


## $\mathbf{\tilde{M}},\mathbf{\tilde{N}}$ expression

In [543]:
# change to CLASS?
def StatisticsLRandomMat(Connparams,Netparams,):
    JE,JI,a,b,g = Connparams
    nn,withrand,wrank,thetatilde = Netparams
    nn = int(nn)
    # #  15112020 reconfinement
    # JE,JI=2.0,1.5#2.9,1.5
    # a,b=0.725,-0.775#0.3,0.2#
    # # with random components or not
    # withrand,wrank,thetatilde = 1.0,1.0,0.6*np.pi

    # calculate the rotation matrix in rank-2 case (reduced), then extend to large 
    # generate perturbed matrix
    J2=np.zeros((2,2))
    J2[:,0],J2[:,1]=JE,-JI
    J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
    J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
    lvec2,svr2,rvech2=la.svd(J2)
    # print('svr2:',svr2)
    M2 = np.array([[1,1],[1,-1]])
    ROT = la.inv(lvec2)@M2/np.sqrt(2) # M@ROT.T/sqrt(2?)=u
    
    eigv2,_=la.eig(J2)

    '''
    Generate two expressions in large random matrix
    params:
        nn = E/I neuron number
        M/Nrank = low rank structure in EI format
        U/V.    = low rank structyre in SVD format
        M/Nranktilde = low rank EI format for large random matrix
        U/Vtilde = low rank SVD format for large random matrix
        sigvmat: diagonal singular values matrix


    '''
    if (a==0) and (b==0):
        grandom=svr2[0]*g
    else:
        grandom=svr2[1]*g#svr2[1]*0.70#g*(svr2[0]+eigv2[0])# for 4case eigv#
    print('g threshold, upper:',2*grandom)
    theta = grandom
    Rc,R = 0,2
    for i in range(2):
        if svr2[i]>grandom:
            Rc+=1
#     nn=600
    sigvmat,rhosigv = np.zeros((Rc,Rc)),np.zeros((Rc,Rc))
    for i in range(Rc):
        sigvmat[i,i]=svr2[i]
        rhosigv[i,i]=svr2[i]+(grandom**2)/svr2[i]
    invsigvmat = la.inv(sigvmat)
    # print('inverse singular value matrix:',invsigvmat)
    # print('square of the inverse singular value matrix:',la.matrix_power(invsigvmat,2)) 

    zetau,zetav = np.random.randn(nn*2,Rc)/np.sqrt(2*nn),np.random.randn(nn*2,Rc)/np.sqrt(2*nn)
    nx,ny = np.random.randn(nn*2,Rc)/np.sqrt(2*nn),np.random.randn(nn*2,Rc)/np.sqrt(2*nn)
    zetau=ny.copy()
    zetav=nx.copy()
    # generate rank-2 matrix P without random entries
    P=np.zeros((nn*2,nn*2))
    P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
    P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
    P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
    # use SVD to decompose the low rank matrix
    lvec,svr,rvech=la.svd(P)
    rvec=rvech.T 
    # check the singular value
    # print(svr,svr2)
    if np.sum((svr[:2]-svr2[:])**2)>1e-8:  
        print('Singular values for 2by2 matrix do not match those for NbyN matrix!')

    '''
        Analytical calculation
    '''
    # diagonal matrix D^-1 and (I-D^-2)^1/2
    # (I-D^-2)^1/2
    alphamat= np.eye(Rc)-(grandom**2)*la.matrix_power(invsigvmat,2)
    Asqrt=np.eye(Rc)
    for i in range(Rc):
        if alphamat[i,i]>0.0:
            Asqrt[i,i]=np.sqrt(alphamat[i,i])
        else:
            Asqrt[i,i]=0.0
            print('AI')
    # print('Asqrt:',Asqrt)
    # D^-1
    randomterm=np.zeros((Rc,Rc))
    for i in range(Rc):
        randomterm[i,i]=np.sqrt(1.0-Asqrt[i,i]**2)

    # Analytically calculate the singular values after transformation(perturbation)
    Alyrho = sigvmat.copy()
    for i in range(Rc):
        Alyrho[i,i]=Alyrho[i,i]+grandom**2/Alyrho[i,i]
        
    # U,V low-rank
    U,V=np.zeros((nn*2,Rc)),np.zeros((nn*2,Rc))
    U,V=lvec[:,:Rc],rvec[:,:Rc]
    # U,V overline (after transformation)
    # U,V zetau,v overline (after transformation)
    Uoverline,Voverline = wrank*U@Asqrt,wrank*V@Asqrt
    zetauoverline,zetavoverline = withrand*zetau@randomterm,withrand*zetav@randomterm
    # U,V tilde (used for reconstruction)
    Utilde,Vtilde = Uoverline+zetauoverline,Voverline+zetavoverline
    Otilde = np.zeros((Rc,Rc))
    if Rc>1:
        # randomly choose a rotation matrix for U,Vtilde and M,Ntilde
        Otilde = ROT#np.array([[np.cos(thetatilde),-np.sin(thetatilde)],[np.sin(thetatilde),np.cos(thetatilde)]])
        # clustring mean after rotation M,N overline
        # from Uoverline and Voverrline (adding analytically calculated singular value)
        Moverline,Noverline = Uoverline@Otilde,Voverline@Alyrho@Otilde

        # change the rotation matrix Otilde
        # the 1st mode of column vector in Moverline is averaging-mode
        # Uup = Uoverline[0,:],Ubottom=Uoverline[-1,:]
        thetam = np.arctan2(U[-1,0],U[0,0])
        gamma = np.arctan2(U[0,0]-U[-1,0],U[0,0]+U[-1,0])
        thetatildeAvg = np.pi/6.0#-gamma#+np.pi/4.0#(U[-1,0]-U[0,0])/(U[0,1]-U[-1,1])
        # tgOtilde = (Uoverline[0,0]-Uoverline[-1,0])/(Uoverline[-1,1]-Uoverline[0,1])
        # thetatildeAvg=-np.arctan(tgOtilde)
        OtildeAvg = ROT#np.array([[np.cos(thetatildeAvg),-np.sin(thetatildeAvg)],[np.sin(thetatildeAvg),np.cos(thetatildeAvg)]])
        # print('rotation:',OtildeAvg)
        print('U:',np.sqrt(nn)*np.array([[U[1,0],U[1,1]],[U[-1,0],U[-1,1]]]))
        rotgamma = np.array([[np.cos(gamma),np.sin(gamma)],[np.sin(gamma),-np.cos(gamma)]])
        D12=np.sum(rotgamma[:,0])*np.sin(thetam)/(np.sum(rotgamma[:,1])*np.cos(thetam))
        print('RATIO:',D12,'org:',Asqrt[0,0]/Asqrt[1,1])
        print('gamma:',np.array([[np.cos(gamma),np.sin(gamma)],[np.sin(gamma),-np.cos(gamma)]]))
        

        # random/perturbation components for M and N, zetamoverline,zetanoverline
        zetamoverline,zetanoverline = zetauoverline@Otilde,zetavoverline@Alyrho@Otilde
        # final expression for M,N tilde
        Mtilde,Ntilde=Moverline+zetamoverline,Noverline+zetanoverline
        Ntildeh = Ntilde.T
    elif Rc==1:
        Moverline,Noverline = Uoverline,Voverline@Alyrho

        # change the rotation matrix Otilde
        # the 1st mode of column vector in Moverline is averaging-mode
        # Uup = Uoverline[0,:],Ubottom=Uoverline[-1,:]
        tgOtilde = 1.0
        OtildeAvg = 1.0

        # random/perturbation components for M and N, zetamoverline,zetanoverline
        zetamoverline,zetanoverline = zetauoverline,zetavoverline@Alyrho
        # final expression for M,N tilde
        Mtilde,Ntilde=Moverline+zetamoverline,Noverline+zetanoverline
        Ntildeh = Ntilde.T       

    '''
        Numerical simulation
    '''
    # add random part in, Xtilde = X+P
    X = (grandom)*np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
    Xtilde=X+P
    Utildenum,sigvxtilde,Vtildenumh=la.svd(Xtilde)
    # the first 2 structural connectivity
    Utildenum,Vtildenum =Utildenum[:,:Rc], Vtildenumh.T
    Vtildenum=Vtildenum[:,:Rc]
    # calculate the mean values of left and right singular vectors U,V numerical overline
    Uoverlinenum,Voverlinenum=Utildenum.copy(),Vtildenum.copy()
    for i in range(Rc):
        Uoverlinenum[:nn,i],Voverlinenum[:nn,i]=np.mean(Uoverlinenum[:nn,i]),np.mean(Voverlinenum[:nn,i])
        Uoverlinenum[nn:,i],Voverlinenum[nn:,i]=np.mean(Uoverlinenum[nn:,i]),np.mean(Voverlinenum[nn:,i])
    # dissect the random components, zetau,v numerical tilde
    zetauoverlinenum,zetavoverlinenum=Utildenum-Uoverlinenum,Vtildenum-Voverlinenum
    if Rc>1:
        MoverlineAvg=Uoverline@OtildeAvg
    elif Rc==1:
        MoverlineAvg=Uoverline


    '''
        properties
    '''
    Ovlzetauvoverline = zetavoverlinenum.T@zetauoverlinenum
    # Overlapuv = Vtildenum.T@Utildenum#sigvmat@Overlapuv@sigvmat/(grandom**2)
    OvlUV = V.T@U
    # calculate the random terms in Utilde and Vtilde
    if Rc>1:
        Xtilde_svdrec = Utilde@Alyrho@Vtilde.T#/grandom**2
        Xtilde_eirec  = Mtilde@Ntildeh#Nranktildeh@Mranktilde#/grandom**2
    else:
        Xtilde_svdrec = Utilde@Alyrho@Vtilde.T#np.reshape(Utilde[:,0],(2*nn,1))*(sigvmat[0,0]+(grandom**2)*invsigvmat[0,0])@np.reshape(Vtilde[:,0],(1,nn*2))#@#/grandom**2
        Xtilde_eirec  = Mtilde@Ntildeh#np.reshape(Nranktildeh[0,:],(1,2*nn))@np.reshape(Mranktilde[:,0],(nn*2,1))#/grandom**2


    # test eigenvalues
    eigv,eigvec=la.eig(Xtilde)
    print('Eigenvalues:',eigv[:R+1])
    _,sigveitilde,_=la.svd(Xtilde_eirec)
    _,sigvsvdtilde,_=la.svd(Xtilde_svdrec)
    NumLRandMat = {'LRandMat':Xtilde,
                   'RandComp':X,
                   'LowRComp':P,
                   'ltilde':Utildenum,
                   'rtilde':Vtildenum,
                   'singularv':sigvxtilde,
                   'lvecmean':Uoverlinenum,
                   'rvecmean':Voverlinenum,
                   'zetalvec':zetauoverlinenum,
                   'zetarvec':zetavoverlinenum
        
    }
    TheosvdLRandMat = {'LRandMat':Xtilde_svdrec,
                   'ltilde':Utilde,
                   'rtilde':Vtilde,
                   'singularv':sigvsvdtilde,
                   'lvecmean':Uoverline,
                   'rvecmean':Voverline,
                   'zetalvec':zetauoverline,
                   'zetarvec':zetavoverline,
                   'Theosingularv':Alyrho 
        
    }
    TheoeiLRandMat = {'LRandMat':Xtilde_eirec,
                   'ltilde':Mtilde,
                   'rtilde':Ntilde,
                   'singularv':sigveitilde,
                   'lvecmean':Moverline,
                   'rvecmean':Noverline,
                   'zetalvec':zetamoverline,
                   'zetarvec':zetanoverline
        
    }

    grandomcomp=np.array([grandom,eigv2[0],svr2[0]])
    return (NumLRandMat,TheosvdLRandMat,TheoeiLRandMat,OvlUV,Ovlzetauvoverline,MoverlineAvg,U,svr2,grandomcomp,Asqrt,Otilde)

## The Statistics of Large Random Matrix
* **Numerical Solution**
* **Theoretical/Analytical Solution**

In [252]:
JE,JI,a,b,g=1.2,1.5,0.6,-0.9,0.7#2.9,1.5,0.3,0.2#
nn,withrand,wrank,thetatilde = 600,1.0,1.0,0.96*np.pi
Connparams=np.array([JE,JI,a,b,g])
Netparams =np.array([nn,withrand,wrank,thetatilde])
NumLRandMat,TheosvdLRandMat,TheoeiLRandMat,OvlUV,Ovlzetauvoverline,MoverlineAvg,U,svr2,grandomcomp,Asqrt=StatisticsLRandomMat(Connparams,Netparams,)
# the overlap between random component *NUMERICAL*
print('RANDOM COMPONENT:')
print(Ovlzetauvoverline)
print('LOWRANK COMPONENT:')
print(OvlUV)

# compare the singular values: numerical, theoretical solutions
print('Singular Values, NUMERICAL:')
print(NumLRandMat['singularv'][:3])
print('NUMERICAL (RECONSTRUCTED)')
print('SVD:',TheosvdLRandMat['singularv'][:3])
print('EI:',TheoeiLRandMat['singularv'][:3])
print('THEORETICAL:')
print(TheosvdLRandMat['Theosingularv'])

# check the rotation matrix for averaging-mode
print('Averaging-mode after Appropriate Rotation:',MoverlineAvg[:3,0],MoverlineAvg[nn:nn+3:,0])
print('2nd mode after Appropriate Rotation:',MoverlineAvg[:3,1],MoverlineAvg[nn:nn+3:,1])

g threshold, upper: 0.16176921899649174
U: [[-0.96285995 -0.27000134]
 [-0.27000134  0.96285995]]
RATIO: -1.0000000000000007 org: 1.3998081072787707
gamma: [[-0.87176457 -0.48992502]
 [-0.48992502  0.87176457]]
Eigenvalues: [0.59985427+0.05972109j 0.59985427-0.05972109j 0.05140341+0.06392436j]
RANDOM COMPONENT:
[[-8.08564689e-06  2.10410830e-04]
 [ 4.60668554e-04  7.41797980e-03]]
LOWRANK COMPONENT:
[[ 0.37139068  0.92847669]
 [-0.92847669  0.37139068]]
Singular Values, NUMERICAL:
[3.11541098 0.17469288 0.16079072]
NUMERICAL (RECONSTRUCTED)
SVD: [3.11525892e+00 1.66946131e-01 2.12111810e-15]
EI: [3.11525892e+00 1.66946131e-01 1.07168929e-15]
THEORETICAL:
[[3.11764934 0.        ]
 [0.         0.17216867]]
Averaging-mode after Appropriate Rotation: [0.03039968 0.03039968 0.03039968] [0.02335916 0.02335916 0.02335916]
2nd mode after Appropriate Rotation: [0.02611415 0.02611415 0.02611415] [-0.01907363 -0.01907363 -0.01907363]


In [125]:
# thetagamma = np.arctan(U[-1,0]/U[0,0])
# print('tan gamma for U:',U[-1,0]/U[0,0])
# thetatilde = thetagamma+np.pi/4.0
# Otilde = np.array([[np.cos(thetatilde),-np.sin(thetatilde)],[np.sin(thetatilde),np.cos(thetatilde)]])
# print('rotation:',Otilde)
# # Uoverline
# g = grandom
# a1,a2=np.sqrt(1-g**2/(svr2[0])**2),np.sqrt(1-g**2/(svr2[1])**2)
# print('Asqwrt',a1,a2)
# Ubar=np.zeros((2,2))
# Ubar[0,0],Ubar[1,0]=a1*U[0,0],a1*U[-1,0]
# Ubar[0,1],Ubar[1,1]=a2*U[0,1],a2*U[-1,1]
# print('U overline here:',Ubar)
# print(TheosvdLRandMat['lvecmean'][:2,0],TheosvdLRandMat['lvecmean'][nn:nn+2,0])
# print(TheosvdLRandMat['lvecmean'][:2,1],TheosvdLRandMat['lvecmean'][nn:nn+2,1])
# Mbar=Ubar@Otilde
# print('M overline meaning:',Mbar)
# print('original Ubar:',Ubar)
# print('sum vec:',Mbar[0,0]**2*nn+Mbar[1,0]**2*nn,Mbar[0,1]**2*nn+Mbar[1,1]**2*nn)

# print('JE/NN,JI/NN,A/NN,B/NN:',JE/nn,JI/nn,a/nn,b/nn)
# # print('JE/NN,JI/NN,A/NN,B/NN:',JE/nn,JI/nn,a/nn,b/nn)

Repfull=TheosvdLRandMat['ltilde']@TheosvdLRandMat['Theosingularv']@TheosvdLRandMat['rtilde'].T
Repbar=TheosvdLRandMat['lvecmean']@TheosvdLRandMat['Theosingularv']@TheosvdLRandMat['rvecmean'].T
eigvfull,_=la.eig(Repfull)
eigvbar,_=la.eig(Repbar)
Numfull=NumLRandMat['LRandMat']
LRConn = NumLRandMat['LowRComp']
eigvnum,_=la.eig(Numfull)
eigvlr,_=la.eig(LRConn)
print('compare the eigenvalue')
print('Real:',eigvnum[:2])
print('lowrank:',eigvlr[:2])
print(r'$\tilde{\mathbf{U}}\Sigma_{\rho}\tilde{\mathbf{V}}^{\intercal}$:',eigvfull[:2])
print(r'$\overline{\mathbf{U}}\Sigma_{\rho}\overline{\mathbf{V}}^{\intercal}$:',eigvbar[:2])
# TheosvdLRandMat = {'LRandMat':Xtilde_svdrec,
#                 'ltilde':Utilde,
#                 'rtilde':Vtilde,
#                 'singularv':sigvsvdtilde,
#                 'lvecmean':Uoverline,
#                 'rvecmean':Voverline,
#                 'Theosingularv'

compare the eigenvalue
Real: [0.70802759+0.j 0.48252917+0.j]
lowrank: [0.6+7.76233761e-08j 0.6-7.76233761e-08j]
$\tilde{\mathbf{U}}\Sigma_{\rho}\tilde{\mathbf{V}}^{\intercal}$: [ 1.15582135e+00+0.00000000e+00j -7.18975324e-17+5.29060262e-17j]
$\overline{\mathbf{U}}\Sigma_{\rho}\overline{\mathbf{V}}^{\intercal}$: [ 1.15706303e+00+0.j -1.40426350e-14+0.j]


# Check the Eigenvalue (Dynamics) under Different Representations

In [554]:
JE,JI,a,b,g=0.8,0.2,0.0,0.0,0.3#2.9,1.5,0.3,0.2#
nn,withrand,wrank,thetatilde = 800,1.0,1.0,0.96*np.pi
Connparams=np.array([JE,JI,a,b,g])
Netparams =np.array([nn,withrand,wrank,thetatilde])
NumLRandMat,TheosvdLRandMat,TheoeiLRandMat,OvlUV,Ovlzetauvoverline,MoverlineAvg,U,svr2,grandomcomp,Asqrt,Otilde=StatisticsLRandomMat(Connparams,Netparams,)

Eigenvalues_collect = np.zeros((nn*2,4),dtype=complex)
# case 1. \tilde{X}=X+P precise expression
Eigenvalues_collect[:,0],_=la.eig(NumLRandMat['LRandMat'])
# case 2. \tilde{U}\rho\tilde{V} approximation 1
Utilde,Vtilde,sigrho_num,sigrho_theo = TheosvdLRandMat['ltilde'],TheosvdLRandMat['rtilde'],TheosvdLRandMat['singularv'],TheosvdLRandMat['Theosingularv']
Eigenvalues_collect[:,1],_=la.eig(Utilde@sigrho_theo@Vtilde.T)
# case 3. \bar{U}\rho\bar{V} approximation 2
Ubar,Vbar = TheosvdLRandMat['lvecmean'],TheosvdLRandMat['rvecmean']
Eigenvalues_collect[:,2],_=la.eig(Ubar@sigrho_theo@Vbar.T)
# case 4. U\sigma\V approximation 3
Eigenvalues_collect[:,3],_=la.eig(NumLRandMat['LowRComp'])

g threshold, upper: 0.6997142273814362
Eigenvalues: [ 0.5902023 +0.j         -0.33490718+0.11758852j -0.33490718-0.11758852j]


In [555]:
plt.rcParams['font.sans-serif'] = ['SimHei'] # 步replace sans-serif 
plt.rcParams['axes.unicode_minus'] = False   

## fontdicts and labels
fonttitle = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 14,
        }
ticks = np.linspace(-1.0,1.0,3)
lims = [-1.1,1.1]

## plot the left singular vectors of rank-2 matrix
fig, ax = plt.subplots(1,2,figsize=(7,7))
ax[0].set_xlim(lims)
ax[0].set_ylim(lims)
ax[0].set_xticks(ticks)
ax[0].set_yticks(ticks)
ax[0].set_aspect('equal')
# ax[0].grid()
ax[1].set_xlim(lims)
ax[1].set_ylim(lims)
ax[1].set_xticks(ticks)
ax[1].set_yticks([])
ax[1].set_aspect('equal')
# ax[1].grid()

print('eigenvalues:',Eigenvalues_collect[0,:])
print('regime:',grandomcomp)
ax[0].scatter(np.real(Eigenvalues_collect[:,0]),np.imag(Eigenvalues_collect[:,0]),s=1.0,c='blue',alpha=0.5)
ax[0].scatter(np.real(Eigenvalues_collect[:,3]),np.imag(Eigenvalues_collect[:,3]),s=20.0,c='red',marker='*')
ax[0].scatter(np.real(Eigenvalues_collect[0,0]),np.imag(Eigenvalues_collect[0,0]),s=20.0,c='blue',alpha=0.5)
ix,iy=np.real(Eigenvalues_collect[0,3]),np.imag(Eigenvalues_collect[0,3])
ax[0].text(ix,iy+0.1,'('+str(format(ix,'.2f'))+','+str(format(iy,'.2f'))+')')

ax[1].scatter(np.real(Eigenvalues_collect[:,1]),np.imag(Eigenvalues_collect[:,1]),s=20.0,c='blue',marker='^')
ax[1].scatter(np.real(Eigenvalues_collect[:,2]),np.imag(Eigenvalues_collect[:,2]),s=20.0,c='red',marker='*')
ix,iy=np.real(Eigenvalues_collect[0,2]),np.imag(Eigenvalues_collect[0,2])
ax[1].text(ix,iy+0.1,'('+str(format(ix,'.2f'))+','+str(format(iy,'.2f'))+')')
ax[1].set_title(r'$J_E,J_I=$'+str(format(JE,'.2f'))+','+str(format(JI,'.2f'))+'\n'+ r'$g,\lambda_{P},\sigma_{P}=$'+str(format(grandomcomp[0],'.2f'))+','+str(format(grandomcomp[1],'.2f'))+','+str(format(grandomcomp[2],'.2f')),fontdict=fonttitle)

eigenvalues: [0.5902023 +0.j 0.60567072+0.j 0.59514   +0.j 0.6       +0.j]
regime: [0.34985711 0.6        1.16619038]


Text(0.5, 1.0, '$J_E,J_I=$0.80,0.20\n$g,\\lambda_{P},\\sigma_{P}=$0.35,0.60,1.17')

In [159]:
JE,JI,a,b,g=1.5,1.2,0.0,-0.0,0.2#2.9,1.5,0.3,0.2#
P=np.zeros((nn*2,nn*2))
P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
lvec2,svr2,rvech2=la.svd(P)
eigv2,_=la.eig(P)
print('eigenvalues and singular values of low rank structure')
print('eigenvalues:',eigv2[:2])
print('singular values:',svr2[:2])
g = (np.real(eigv2[0])+svr2[0])*0.5
Testrand = g*np.random.randn(nn*2,nn*2)/np.sqrt(nn*2)
# eigvr,_=la.eig(Testrand)
# _,svdr,_=la.svd(Testrand)
# fig=plt.figure()
# ax1=fig.add_subplot(211)
# ax2=fig.add_subplot(212)
# n, bins, patches = ax1.hist(svdr, 100, density=True, alpha=0.5)
# ax2.scatter(np.real(eigvr),np.imag(eigvr))
# perturbation
xtd = P+Testrand
eigvp,_=la.eig(xtd)
_,svdvp,_=la.svd(xtd)
print('random component:',g)
print('eigenvalues and singular values comparison:')
print('eigenvalues:',eigvp[:2])
print('singular values:',svdvp[:2])

print('theoretical:',np.sqrt(2*(JE**2+JI**2)),(JE-JI))
print('theoretical after:',(JE-JI)*(1-g**4/svr2[0]**4))




eigenvalues and singular values of low rank structure
eigenvalues: [ 1.40000000e+00+0.j -4.75912847e-16+0.j]
singular values: [2.77848880e+00 3.55580407e-14]
random component: 2.0892443989449814
eigenvalues and singular values comparison:
eigenvalues: [1.61431552+1.32358205j 1.61431552-1.32358205j]
singular values: [4.37714879 4.17469307]
theoretical: 2.778488797889961 1.4
theoretical after: 0.9524408697859066


# Other details 
* checking the distribution of the first half and the remaining
* checking the effect of rotation $\to$ changing the distribution? or just rotate the distribution

In [354]:
JE,JI,a,b,g=1.2,1.5,0.6,-0.9,0.5#2.9,1.5,0.3,0.2#
nn,withrand,wrank,thetatilde = 600,1.0,1.0,0.96*np.pi
Connparams=np.array([JE,JI,a,b,g])
Netparams =np.array([nn,withrand,wrank,thetatilde])
NumLRandMat,TheosvdLRandMat,TheoeiLRandMat,OvlUV,Ovlzetauvoverline,MoverlineAvg,U,svr2,grandomcomp,Asqrt=StatisticsLRandomMat(Connparams,Netparams,)

g threshold, upper: 0.11554944214035126
U: [[-0.96285995 -0.27000134]
 [-0.27000134  0.96285995]]
RATIO: -1.0000000000000007 org: 1.154501982325283
gamma: [[-0.87176457 -0.48992502]
 [-0.48992502  0.87176457]]
Eigenvalues: [ 0.69344664+0.j          0.51235687+0.j         -0.04195045+0.03996108j]


In [291]:
Utildenum,Vtildenum=NumLRandMat['ltilde'],NumLRandMat['rtilde']
Ubarnum,Vbarnum=NumLRandMat['lvecmean'],NumLRandMat['rvecmean']
# histograms of random components in U_NE and U_NI
fig, ax = plt.subplots(figsize=(7,7))
ax.scatter(Ubarnum[:nn,0],Ubarnum[nn:,0],s=10.0,c='red',marker='^')
ax.scatter(Ubarnum[:nn,1],Ubarnum[nn:,1],s=10.0,c='red',marker='^')
xnorm = 1.0/np.sqrt(nn)
ax.set_xlim([-xnorm,xnorm])
ax.set_ylim([-xnorm,xnorm])
ax.set_aspect('equal')


Utilderand, Vtilderand=np.zeros_like(Utildenum),np.zeros_like(Vtildenum)
Utilderand,Vtilderand=Utildenum-Ubarnum,Vtildenum-Vbarnum
fig, ax = plt.subplots(2,2,figsize=(7,7))
i=0 # U or V
for j in range(2):
    ax[i][j].hist(Utilderand[:nn,j],50,density=False,facecolor='r',alpha=0.25)
    ax[i][j].hist(Utilderand[nn:,j],50,density=False,facecolor='b',alpha=0.25)
    if j<1:
        start, end = -0.003,0.0031
        ax[i][j].set_xlim([start,end])
        ax[i][j].set_ylim([0,50])
        ax[i][j].xaxis.set_ticks(np.arange(start, end,0.003)) 
        ax[i][j].yaxis.tick_left()
        ax[i][j].set_aspect('auto')
        ax[i][j].set_title('Left Noise Components')
    else:
        start, end = -0.06,0.061
        ax[i][j].set_xlim([start,end])
        ax[i][j].set_ylim([0,50])
        ax[i][j].xaxis.set_ticks(np.arange(start, end,0.03)) 
        ax[i][j].yaxis.tick_right()
        ax[i][j].set_aspect('auto')
        ax[i][j].set_title('Left Noise Components')
i=1 # U or V
for j in range(2):
    ax[i][j].hist(Vtilderand[:nn,j],50,density=False,facecolor='r',alpha=0.25)
    ax[i][j].hist(Vtilderand[nn:,j],50,density=False,facecolor='b',alpha=0.25)
    if j<1:
        start, end = -0.003,0.0031
        ax[i][j].set_xlim([start,end])
        ax[i][j].set_ylim([0,50])
        ax[i][j].xaxis.set_ticks(np.arange(start, end,0.003)) 
        ax[i][j].yaxis.tick_left()
        ax[i][j].set_aspect('auto')
        ax[i][j].set_title('Right Noise Components')
    else:
        start, end = -0.06,0.061
        ax[i][j].set_xlim([start,end])
        ax[i][j].set_ylim([0,50])
        ax[i][j].xaxis.set_ticks(np.arange(start, end,0.03)) 
        ax[i][j].yaxis.tick_right()
        ax[i][j].set_aspect('auto')
        ax[i][j].set_title('Right Noise Components')

# whether there are correlations
corrEIU, corrEIV = Utilderand[:nn,:].T@Utilderand[nn:,:],Vtilderand[:nn,:].T@Vtilderand[nn:,:]
print('correlation between E and I arrays:',corrEIU,'V',corrEIV)

correlation between E and I arrays: [[ 1.34793127e-06 -9.58440167e-04]
 [-1.61886246e-04  1.87305784e-02]] V [[-1.24425735e-05  7.63628696e-05]
 [ 1.52781814e-04 -2.22458785e-03]]


# How the Rotation Changes the Representation
* the coordinate system $\bar{\mathbf{U}}=\mathbf{U}\sqrt{1-g^2/\sigma^2}$ (projecting to 2D plane)
* the coordinate system $\bar{\mathbf{M}}=\mathbf{U}\sqrt{1-g^2/\sigma^2}\mathbf{\tilde{O}}$


In [454]:
fonttitle = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }

In [481]:
JE,JI,a,b,g=1.2,1.5,0.6,-0.9,0.9#2.9,1.5,0.3,0.2#
nn,withrand,wrank,thetatilde = 600,1.0,1.0,np.pi/4.0
Connparams=np.array([JE,JI,a,b,g])
Netparams =np.array([nn,withrand,wrank,thetatilde])
NumLRandMat,TheosvdLRandMat,TheoeiLRandMat,OvlUV,Ovlzetauvoverline,MoverlineAvg,U,svr2,grandomcomp,Asqrt,Otilde=StatisticsLRandomMat(Connparams,Netparams,)

plt.rcParams['font.sans-serif'] = ['SimHei'] # 步replace sans-serif 
plt.rcParams['axes.unicode_minus'] = False   

## plot the left singular vectors of rank-2 matrix
fig, ax = plt.subplots(1,2,figsize=(7,7))
ax[0].set_xlim([-0.051,0.051])
ax[0].set_ylim([-0.051,0.051])
ax[0].set_aspect('equal')
ax[1].set_xlim([-0.051,0.051])
ax[1].set_ylim([-0.051,0.051])
ax[1].set_aspect('equal')
# # ax[0].arrow([0,0],[0,0],[U[0,0],U[0,1]],[U[-1,0],U[-1,1]])
# ax[0].arrow(0,0,U[0,0],U[-1,0],color='b',alpha=0.25,label=r'$\mathbf{U}$')#,head_width=0.2)
# ax[0].arrow(0,0,U[0,1],U[-1,1],color='b',alpha=0.25)

## use numerical solutions
Utildenum,Vtildenum=NumLRandMat['ltilde'],NumLRandMat['rtilde']
Ubarnum,Vbarnum=NumLRandMat['lvecmean'],NumLRandMat['rvecmean']
# # histograms of random components in U_NE and U_NI
# ax[0].scatter(Ubarnum[:nn,0],Ubarnum[nn:,0],s=10.0,c='red',marker='^',label=r'numerical $\bar{\mathbf{U}}$')
# ax[0].scatter(Ubarnum[:nn,1],Ubarnum[nn:,1],s=10.0,c='red',marker='^')

## use theoretical solutions
Utildetheo,Vtildetheo=TheosvdLRandMat['ltilde'],TheosvdLRandMat['rtilde']
Ubartheo,Vbartheo=TheosvdLRandMat['lvecmean'],TheosvdLRandMat['rvecmean']
# histograms of random components in U_NE and U_NI
# ax[0].arrow([0,0],[0,0],[Ubartheo[0,0],Ubartheo[0,1]],[Ubartheo[-1,0],Ubartheo[-1,1]])
ax[0].arrow(0,0,Ubartheo[0,0],Ubartheo[-1,0],color='b',alpha=1.0,label=r'theoretical $\bar{\mathbf{U}}$')#,head_width=0.2)
ax[0].arrow(0,0,Ubartheo[0,1],Ubartheo[-1,1],color='b',alpha=0.5)

zetauvec=np.zeros((2,2))
zetauvec[:,0],zetauvec[:,1]=[Ubartheo[0,0],Ubartheo[-1,0]],[Ubartheo[0,1],Ubartheo[-1,1]]
zetauvec[:,0],zetauvec[:,1]=zetauvec[:,0]/np.sqrt(np.sum(zetauvec[:,0]**2)),zetauvec[:,1]/np.sqrt(np.sum(zetauvec[:,1]**2))
zetauvec[:,0],zetauvec[:,1]=zetauvec[:,0]*grandomcomp[0]/svr2[0]/np.sqrt(2*nn),zetauvec[:,1]*grandomcomp[0]/svr2[1]/np.sqrt(2*nn)
ax[0].arrow(Ubartheo[0,1],Ubartheo[-1,1],zetauvec[0,0],zetauvec[-1,0],color='r',alpha=1.0)
ax[0].arrow(Ubartheo[0,1],Ubartheo[-1,1],zetauvec[0,1],zetauvec[-1,1],color='r',alpha=0.5)
ax[0].text(0,0.03,'noise \n'+r'components $\zeta^{\bar{\mathbf{U}}}$')
ax[0].text(-0.02,-0.025,'structural \n'+r'components $\bar{\mathbf{U}}$')
# setting ax ticks and label
xticks=np.linspace(-0.05,0.05,3)
ax[0].set_xticks(xticks)
ax[0].set_yticks(xticks)
ax[0].set_title('coordinate '+r'$\bar{\mathbf{U}}$',fontdict=fonttitle)
# calculate the random terms
Rc=2
randomterm=np.zeros((Rc,Rc))
for i in range(Rc):
    randomterm[i,i]=np.sqrt(1.0-Asqrt[i,i]**2)
zetau,zetav = np.random.randn(nn*2,Rc)/np.sqrt(2*nn),np.random.randn(nn*2,Rc)/np.sqrt(2*nn)
zetauoverline ,zetavoverline = zetau@randomterm,zetav@randomterm

# zetauoverlineE,zetauoverlineI=zetauoverline[:nn,:],zetauoverline[nn:,:]
# bulkuE,bulkuI=zetauoverlineE+Ubartheo[:nn,:],zetauoverlineI+Ubartheo[nn:,:]
# # ax[0].scatter(bulkuE,bulkuI,s=5.0,c='red',marker='*',alpha=0.25)
# circle1 = plt.Circle((Ubartheo[0,0], Ubartheo[-1,0]), 1/np.sqrt(2*nn)*grandomcomp[0]/svr2[0],color='r',alpha=0.25)
# ax[0].add_artist(circle1)
# circle2 = plt.Circle((Ubartheo[0,1], Ubartheo[-1,1]), 1/np.sqrt(2*nn)*grandomcomp[0]/svr2[1],color='r',alpha=0.25)
# ax[0].add_artist(circle2)




g threshold, upper: 0.20798899585263228
U: [[-0.96285995 -0.27000134]
 [-0.27000134  0.96285995]]
RATIO: -1.0000000000000007 org: 2.292878941968175
gamma: [[-0.87176457 -0.48992502]
 [-0.48992502  0.87176457]]
Eigenvalues: [0.59514426+0.02139804j 0.59514426-0.02139804j 0.05697013+0.08875873j]


In [482]:
## use theoretical solutions
Mtildetheo,Ntildetheo=TheoeiLRandMat['ltilde'],TheoeiLRandMat['rtilde']
Mbartheo,Nbartheo=TheoeiLRandMat['lvecmean'],TheoeiLRandMat['rvecmean']
# histograms of random components in U_NE and U_NI
# ax[0].arrow([0,0],[0,0],[Ubartheo[0,0],Ubartheo[0,1]],[Ubartheo[-1,0],Ubartheo[-1,1]])

## for comparison
ax[1].arrow(0,0,Ubartheo[0,0],Ubartheo[-1,0],linestyle='-.',color='b',alpha=0.5)
ax[1].arrow(0,0,Ubartheo[0,1],Ubartheo[-1,1],linestyle='-.',color='b',alpha=0.25)
ax[1].arrow(Ubartheo[0,1],Ubartheo[-1,1],zetauvec[0,0],zetauvec[-1,0],linestyle='-.',color='r',alpha=0.5)
ax[1].arrow(Ubartheo[0,1],Ubartheo[-1,1],zetauvec[0,1],zetauvec[-1,1],linestyle='-.',color='r',alpha=0.25)

## should have after rotation
ax[1].arrow(0,0,Mbartheo[0,0],Mbartheo[-1,0],color='b',alpha=1.0)#,alpha=0.25,label=r'theoretical $\bar{\mathbf{M}}$')#,head_width=0.2)
ax[1].arrow(0,0,Mbartheo[0,1],Mbartheo[-1,1],color='b',alpha=0.5)#,alpha=0.25)

zetamvec= zetauvec@Otilde
ax[1].arrow(Mbartheo[0,0],Mbartheo[-1,0],zetamvec[0,0],zetamvec[-1,0],linestyle='-.',color='r',alpha=1.0)
# ax[1].arrow(Mbartheo[0,1],Mbartheo[-1,1],zetamvec[0,1],zetamvec[-1,1],color='r',alpha=0.5)

ax[1].arrow(Mbartheo[0,1],Mbartheo[-1,1],zetamvec[0,0],zetamvec[-1,0],color='r',alpha=1.0)
ax[1].arrow(Mbartheo[0,1],Mbartheo[-1,1],zetamvec[0,1],zetamvec[-1,1],color='r',alpha=0.5)

ax[1].arrow(0,0,Mbartheo[0,0]+zetamvec[0,0],Mbartheo[-1,0]+zetamvec[-1,0],color='g',alpha=1.0)#,alpha=0.25)
ax[1].arrow(0,0,Mbartheo[0,1]+zetamvec[0,1],Mbartheo[-1,1]+zetamvec[-1,1],color='g',alpha=0.5)
testortho=(Mbartheo[0,0]+zetamvec[0,0])*(Mbartheo[0,1]+zetamvec[0,1])+(Mbartheo[-1,0]+zetamvec[-1,0])*(Mbartheo[-1,1]+zetamvec[-1,1])
print(testortho)
vecsum=np.zeros((2,2))
vecsum[:,0]=[Mbartheo[0,0]+zetamvec[0,0],Mbartheo[-1,0]+zetamvec[-1,0]]
vecsum[:,1]=[Mbartheo[0,1]+zetamvec[0,1],Mbartheo[-1,1]+zetamvec[-1,1]]
print(vecsum[:,0],vecsum[:,1])
print(Mbartheo[:,0],Mbartheo[:,1])
# ax[1].text(0,0.03,'noise \n'+r'components $\zeta^{\bar{\mathbf{U}}}$')

# setting ax ticks and label
ax[1].set_xticks(xticks)
ax[1].set_yticks(xticks)
ax[1].set_title('coordinate '+r'$\bar{\mathbf{M}}$',fontdict=fonttitle)



-7.344353287834179e-05
[0.02926686 0.03048107] [ 0.03000594 -0.03122015]
[0.0318948  0.0318948  0.0318948  ... 0.01799838 0.01799838 0.01799838] [ 0.02343611  0.02343611  0.02343611 ... -0.00953969 -0.00953969
 -0.00953969]


Text(0.5, 1.0, 'coordinate $\\bar{\\mathbf{M}}$')

# Rotation and Distribution 

In [536]:
## fontdicts and labels
fonttitle = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 14,
        }
ticksrandom = np.linspace(-0.1,0.1,3)
ticksstruct = np.linspace(-1.0,1.0,3)
limrandom = [-0.12,0.12]
limstruct = [-1.5,1.5]
## network settings 
JE,JI,a,b,g=1.2,1.5,0.6,-0.9,0.9#2.9,1.5,0.3,0.2#
nn,withrand,wrank,thetatilde = 600,1.0,1.0,np.pi/4.0
## function variables
Connparams=np.array([JE,JI,a,b,g])
Netparams =np.array([nn,withrand,wrank,thetatilde])
# NumLRandMat,TheosvdLRandMat,TheoeiLRandMat,OvlUV,Ovlzetauvoverline,MoverlineAvg,U,svr2,grandomcomp,Asqrt,Otilde=StatisticsLRandomMat(Connparams,Netparams,)

In [538]:
## for only structural connectivity
J2=np.zeros((2,2))
J2[:,0],J2[:,1]=JE,-JI
J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
lvec2,svr2,rvech2=la.svd(J2)
M = np.array([[1,1],[1,-1]])
ROT = la.inv(lvec2)@M/np.sqrt(2) 

# rotation
Mhat = np.array([[1,1],[1,-1]])
Nhat = np.array([[JE,a],[-JI,b]])
# svd decomposition
Morg = lvec2.copy()
Norg = rvech2.T
Norg[:,0]*=svr2[0]
Norg[:,1]*=svr2[1]

fig, ax = plt.subplots(1,2,figsize=(7,7))
colorlist ='br'
for inn in range(2):
    ax[0].plot([0,Morg[inn,0]],[0,Morg[inn,1]],colorlist[inn],linewidth=2.0,alpha=0.5)#,label=r'$(m_1^{(1)},m_1^{(2)})$')
    ax[0].plot([0,Mhat[inn,0]],[0,Mhat[inn,1]],colorlist[inn],linewidth=2.0,alpha=1.0)#,label=r'$(\hat{m}_1^{(1)},\hat{m}_1^{(2)}$)')
    ax[0].arrow(Morg[inn,0],Morg[inn,1],Mhat[inn,0]-Morg[inn,0],Mhat[inn,1]-Morg[inn,1],linestyle='--',color='black',alpha=0.25,head_width=0.12)

# ax[0].plot([0,Morg[1,0]],[0,Morg[1,1]],'r',alpha=0.5,label=r'$(m_2^{(1)},m_2^{(2)}$')
# ax[0].plot([0,Mhat[1,0]],[0,Mhat[1,1]],'r',label=r'$(\hat{m}_2^{(1)},\hat{m}_2^{(2)}$')
# ax[0].arrow(Morg[1,0],Morg[1,1],Mhat[1,0]-Morg[1,0],Mhat[1,1]-Morg[1,1],linestyle='--',color='black',alpha=0.25,head_width=0.08)
ax[0].set_xlim(limstruct)
ax[0].set_ylim(limstruct)
ax[0].set_xticks(ticksstruct)
ax[0].set_yticks(ticksstruct)
ax[0].set_aspect('equal')
ax[0].grid()
# ax[0].legend()
ax[0].set_title('loading vectors \n'+r'$(m_i^{(1)},m_i^{(2)})\to(\hat{m}_i^{(1)},\hat{m}_i^{(2)}$)',fontdict=fonttitle)

for inn in range(2):
    ax[1].plot([0,Norg[inn,0]],[0,Norg[inn,1]],colorlist[inn],linewidth=2.0,alpha=0.5)#,label=r'$(n_1^{(1)},n_1^{(2)})$')
    ax[1].plot([0,Nhat[inn,0]],[0,Nhat[inn,1]],colorlist[inn],linewidth=2.0,alpha=1.0)#,label=r'$(\hat{n}_1^{(1)},\hat{n}_1^{(2)}$)')
    ax[1].arrow(Norg[inn,0],Norg[inn,1],Nhat[inn,0]-Norg[inn,0],Nhat[inn,1]-Norg[inn,1],linestyle='--',color='black',alpha=0.25,head_width=0.12)

# ax[1].plot([0,Norg[1,0]],[0,Norg[1,1]],'r',alpha=0.25,label=r'$(n_2^{(1)},n_2^{(2)}$')
# ax[1].plot([0,Nhat[1,0]],[0,Nhat[1,1]],'r',label=r'$(\hat{n}_2^{(1)},\hat{n}_2^{(2)}$')
# ax[1].arrow(Norg[1,0],Norg[1,1],Nhat[1,0]-Norg[1,0],Nhat[1,1]-Norg[1,1],linestyle='--',color='black',alpha=0.25,head_width=0.08)
limstruct_=[-3.0,3.0]
ticksstruct_=np.linspace(-2.0,2.0,3)
ax[1].set_xlim(limstruct_)
ax[1].set_ylim(limstruct_)
ax[1].set_xticks(ticksstruct_)
ax[1].set_yticks(ticksstruct_)
ax[1].set_aspect('equal')
ax[1].grid()
# ax[1].legend()
ax[1].set_title('loading vectors \n'+r'$(n_i^{(1)},n_i^{(2)})\to(\hat{n}_i^{(1)},\hat{n}_i^{(2)}$)',fontdict=fonttitle)
 

Text(0.5, 1.0, 'loading vectors \n$(n_i^{(1)},n_i^{(2)})\\to(\\hat{n}_i^{(1)},\\hat{n}_i^{(2)}$)')

In [525]:
## use theoretical solutions
Utildetheo,Vtildetheo=TheosvdLRandMat['ltilde'],TheosvdLRandMat['rtilde']
sigvrho = TheosvdLRandMat['Theosingularv']
V_tildetheo=Vtildetheo@sigvrho
Ubartheo,Vbartheo=TheosvdLRandMat['lvecmean'],TheosvdLRandMat['rvecmean']
V_bartheo = Vbartheo@sigvrho
## use theoretical solutions
Mtildetheo,Ntildetheo=TheoeiLRandMat['ltilde'],TheoeiLRandMat['rtilde']
Mbartheo,Nbartheo=TheoeiLRandMat['lvecmean'],TheoeiLRandMat['rvecmean']
## for random connectivity
fig, ax = plt.subplots(1,2,figsize=(7,7))
colorlist ='br'
for inn in range(2):
    ax[0].plot([0,Ubartheo[(inn)*nn,0]],[0,Ubartheo[(inn)*nn,1]],colorlist[inn],linewidth=1.5,alpha=0.5)#,label=r'$(m_1^{(1)},m_1^{(2)})$')
    ax[0].plot([0,Mbartheo[(inn)*nn,0]],[0,Mbartheo[(inn)*nn,1]],colorlist[inn],linewidth=1.5,alpha=1.0)#,label=r'$(\hat{m}_1^{(1)},\hat{m}_1^{(2)}$)')
    ax[0].arrow(Ubartheo[(inn)*nn,0],Ubartheo[(inn)*nn,1],Mbartheo[(inn)*nn,0]-Ubartheo[(inn)*nn,0],Mbartheo[(inn)*nn,1]-Ubartheo[(inn)*nn,1],linestyle='--',color='black',alpha=0.25)#,head_width=0.12)
# scatter the random large matrix
ax[0].scatter(Utildetheo[:,0],Utildetheo[:,1],s=1.0,c='green',alpha=0.2)
ax[0].scatter(Mtildetheo[:,0],Mtildetheo[:,1],s=1.0,c='green',alpha=0.8)
ax[0].set_xlim(limrandom)
ax[0].set_ylim(limrandom)
ax[0].set_xticks(ticksrandom)
ax[0].set_yticks(ticksrandom)
ax[0].set_aspect('equal')
ax[0].grid()
ax[0].set_title('loading vectors \n'+r'$(\tilde{u}_i^{(1)},\tilde{u}_i^{(2)})\to(\tilde{m}_i^{(1)},\tilde{m}_i^{(2)}$)',fontdict=fonttitle)

for inn in range(2):
    ax[1].plot([0,V_bartheo[(inn)*nn,0]],[0,V_bartheo[(inn)*nn,1]],colorlist[inn],linewidth=1.5,alpha=0.5)#,label=r'$(m_1^{(1)},m_1^{(2)})$')
    ax[1].plot([0,Nbartheo[(inn)*nn,0]],[0,Nbartheo[(inn)*nn,1]],colorlist[inn],linewidth=1.5,alpha=1.0)#,label=r'$(\hat{m}_1^{(1)},\hat{m}_1^{(2)}$)')
    ax[1].arrow(V_bartheo[(inn)*nn,0],V_bartheo[(inn)*nn,1],Nbartheo[(inn)*nn,0]-V_bartheo[(inn)*nn,0],Nbartheo[(inn)*nn,1]-V_bartheo[(inn)*nn,1],linestyle='--',color='black',alpha=0.25)#,head_width=0.12)
# scatter the random large matrix
ax[1].scatter(V_tildetheo[:,0],V_tildetheo[:,1],s=1.0,c='green',alpha=0.2)
ax[1].scatter(Ntildetheo[:,0],Ntildetheo[:,1],s=1.0,c='green',alpha=0.8)
ax[1].set_xlim(limrandom)
ax[1].set_ylim(limrandom)
ax[1].set_xticks(ticksrandom)
ax[1].set_yticks(ticksrandom)
ax[1].set_aspect('equal')
ax[1].grid()
ax[1].set_title('loading vectors \n'+r'$(\tilde{v}_i^{(1)}\rho_1,\tilde{v}_i^{(2)}\rho_2)\to(\tilde{n}_i^{(1)},\tilde{n}_i^{(2)}$)',fontdict=fonttitle)

Text(0.5, 1.0, 'loading vectors \n$(\\tilde{v}_i^{(1)}\\rho_1,\\tilde{v}_i^{(2)}\\rho_2)\\to(\\tilde{n}_i^{(1)},\\tilde{n}_i^{(2)}$)')

# General Rotation Case

In [2]:
'''
    Rank-2 structure and 2 populations
    (m_1^E,m_2^E,n_1^E,n_2^E)
    (m_1^I,m_2^I,n_1^I,n_2^I)
    s.t. m^1^E*m^2^E+~I=0, also n
'''
# for left singular vectors -- two popuplations
NE,NI,R,P=10000,10000,2,2
minp,nout={},{}
minp['E'],minp['I']=np.zeros((NE,R)),np.zeros((NI,R))
nout['E'],nout['I']=np.zeros((NE,R)),np.zeros((NI,R))
## for minp a1E*a2E+a1I*a2I=0
# vectors (a1E,a1I), (a2E,a2I) are orthogonal, assume 
# zeta 1 and zeta 2 are independent? (or zeta1,2,E,Ialso ortho)
# a1E^2+sigma1E^2+for I = 
# statistics of left 
a1E,a1I,a2E=0.3/np.sqrt(NE),0.6/np.sqrt(NI),0.8/np.sqrt(NE)
a2I = -a1E*a2E/a1I
sigmaa1E,sigmaa1I,sigmaa2E,sigmaa2I=np.sqrt(1/NE-a1E**2),np.sqrt(1/NI-a1I**2),np.sqrt(1/NE-a2E**2),np.sqrt(1/NI-a2I**2)
zetamE1,zetamE2,zetamI1,zetamI2=np.random.randn(NE),np.random.randn(NE),np.random.randn(NI),np.random.randn(NI)
minp['E'][:,0],minp['E'][:,1]=a1E+sigmaa1E*zetamE1,a2E+sigmaa2E*zetamE2
minp['I'][:,0],minp['I'][:,1]=a1I+sigmaa1I*zetamI1,a2I+sigmaa2I*zetamI2
print('orthogonal:',np.sum(minp['E'][:,0]*minp['E'][:,1])+np.sum(minp['I'][:,0]*minp['I'][:,1]))
print('normalization, 1:',np.sum(minp['E'][:,0]**2)+np.sum(minp['I'][:,0]**2))
print('normalization, 2:',np.sum(minp['E'][:,1]**2)+np.sum(minp['I'][:,1]**2))

# define the overlap matrix
# overlap matrix should be finely designed
ovlorder=np.arange(2,10,2)*0.1
sigmavuE=np.reshape(ovlorder,(2,2))
sigmavuE[1,1]=-sigmavuE[0,0]*sigmavuE[1,0]/sigmavuE[0,1]
sigmavuI=sigmavuE.copy()
sigmavuI=sigmavuI.T
sigmavuI[0,0],sigmavuI[1,1]=sigmavuI[1,1],sigmavuI[0,0]
sigv=np.linspace(1.6,0.8,R)
b1E,b1I,b2E=0.4/np.sqrt(NE),0.6/np.sqrt(NI),-0.2/np.sqrt(NE)
b2I = -b1E*b2E/b1I
sigmab1E,sigmab1I,sigmab2E,sigmab2I=np.sqrt(1/NE-b1E**2),np.sqrt(1/NI-b1I**2),np.sqrt(1/NE-b2E**2),np.sqrt(1/NI-b2I**2)
zetanE1,zetanE2,zetanI1,zetanI2=np.random.randn(NE),np.random.randn(NE),np.random.randn(NI),np.random.randn(NI)
nout['E'][:,0]=b1E+sigmab1E*(np.sqrt(1-sigmavuE[0,1]**2-sigmavuE[0,0]**2)*zetanE1+zetamE1*sigmavuE[0,0]+zetamE2*sigmavuE[0,1])
nout['E'][:,1]=b2E+sigmab2E*(np.sqrt(1-sigmavuE[1,1]**2-sigmavuE[1,0]**2)*zetanE2+zetamE1*sigmavuE[1,0]+zetamE2*sigmavuE[1,1])

nout['I'][:,0]=b1I+sigmab1I*(np.sqrt(1-sigmavuI[0,1]**2-sigmavuI[0,0]**2)*zetanI1+zetamI1*sigmavuI[0,0]+zetamI2*sigmavuI[0,1])
nout['I'][:,1]=b2I+sigmab2I*(np.sqrt(1-sigmavuI[1,1]**2-sigmavuI[1,0]**2)*zetanI2+zetamI1*sigmavuI[1,0]+zetamI2*sigmavuI[1,1])
print('orthogonal:',np.sum(nout['E'][:,0]*nout['E'][:,1])+np.sum(nout['I'][:,0]*nout['I'][:,1]))
print('normalization, 1:',np.sum(nout['E'][:,0]**2)+np.sum(nout['I'][:,0]**2))
print('normalization, 2:',np.sum(nout['E'][:,1]**2)+np.sum(nout['I'][:,1]**2))
for i in range(R):
    nout['E'][:,i]*=sigv[i]
    nout['I'][:,i]*=sigv[i]
print(sigv)

orthogonal: 0.006851002648033094
normalization, 1: 1.9820404437713224
normalization, 2: 1.9697573537678486
orthogonal: -0.008060556880316552
normalization, 1: 1.9625184328858527
normalization, 2: 1.9988440342541622
[1.6 0.8]


In [3]:
# theoretical overlap matrix
# EXC
zetacompenE=np.zeros((2,2))
zetacompenE=np.array([[sigmab1E*sigmavuE[0,0]*sigmaa1E,sigmab1E*sigmavuE[0,1]*sigmaa2E],[sigmab2E*sigmavuE[1,0]*sigmaa1E,sigmab2E*sigmavuE[1,1]*sigmaa2E]])
avgcompenE=np.array([[b1E*a1E,b1E*a2E],[b2E*a1E,b2E*a2E]])
ttheonmE=(avgcompenE+zetacompenE)*NE
ttheonmE[0,:]*=sigv[0]
ttheonmE[1,:]*=sigv[1]
print('Excitatory theo overlap')
print(ttheonmE)

# INH
zetacompenI=np.array([[sigmab1I*sigmavuI[0,0]*sigmaa1I,sigmab1I*sigmavuI[0,1]*sigmaa2I],[sigmab2I*sigmavuI[1,0]*sigmaa1I,sigmab2I*sigmavuI[1,1]*sigmaa2I]])
avgcompenI=np.array([[b1E*a1E,b1E*a2E],[b2E*a1E,b2E*a2E]])
ttheonmE=(avgcompenE+zetacompenE)*NE
ttheonmE[0,:]*=sigv[0]
ttheonmE[1,:]*=sigv[1]
print('Excitatory theo overlap')
print(ttheonmE)




Excitatory theo overlap
[[ 0.47177591  0.86394181]
 [ 0.40063954 -0.26909061]]
Excitatory theo overlap
[[ 0.47177591  0.86394181]
 [ 0.40063954 -0.26909061]]


In [40]:
# supporting info
am,an={},{}
for ipop,iname in enumerate('EI'):
    if iname=='E':
        am[iname],an[iname]=[],[]
        am[iname]=np.array([a1E,a2E])
        an[iname]=np.array([b1E,b2E])
    elif iname=='I':
        am[iname],an[iname]=[],[]
        am[iname]=np.array([a1I,a2I])
        an[iname]=np.array([b1I,b2I])  
# we have sigmavuE,sigmavuI  calculate sigma
sigmam,sigman={},{}
for ipop,iname in enumerate('EI'):
    if iname=='E':
        sigmam[iname],sigman[iname]=[],[]
        sigmam[iname]=np.array([sigmaa1E,sigmaa2E])
        sigman[iname]=np.array([sigmab1E,sigmab2E])
    elif iname=='I':
        sigmam[iname],sigman[iname]=[],[]
        sigmam[iname]=np.array([sigmaa1I,sigmaa2I])
        sigman[iname]=np.array([sigmab1I,sigmab2I])  


In [25]:
print('all the variables:')
print('a-averaging, m',am)
print('n:',an)
print('all the standard deviations, m',sigmam)
print('n',sigman)
print('projection angle q, E',sigmavuE)
print('I',sigmavuI)
## test orthogonal and unitarity
print('orthogonal, M',np.sum(minp['E'][:,0]*minp['E'][:,1])+np.sum(minp['I'][:,0]*minp['I'][:,1]))
print('orthogonal, N',np.sum(nout['E'][:,0]*nout['E'][:,1])+np.sum(nout['I'][:,0]*nout['I'][:,1]))

print('orthogonal, M',np.sum(minp['E'][:,0]**2)+np.sum(minp['I'][:,0]**2),np.sum(minp['E'][:,1]**2)+np.sum(minp['I'][:,1]**2))
print('orthogonal, N',np.sum(nout['E'][:,0]**2)/sigv[0]**2+np.sum(nout['I'][:,0]**2)/sigv[0]**2,np.sum(nout['E'][:,1]**2)/sigv[1]**2+np.sum(nout['I'][:,1]**2)/sigv[1]**2)


all the variables:
a-averaging, m {'E': array([0.003, 0.008]), 'I': array([ 0.006, -0.004])}
n: {'E': array([ 0.004, -0.002]), 'I': array([0.006     , 0.00133333])}
all the standard deviations, m {'E': array([0.00953939, 0.006     ]), 'I': array([0.008     , 0.00916515])}
n {'E': array([0.00916515, 0.00979796]), 'I': array([0.008     , 0.00991071])}
projection angle q, E [[ 0.2  0.4]
 [ 0.6 -0.3]]
I [[-0.3  0.6]
 [ 0.4  0.2]]
orthogonal, M 0.00301650282963678
orthogonal, N 0.00720004548145875
orthogonal, M 2.000774353373153 2.000197733655458
orthogonal, N 1.9913091484647358 2.013138008109327


## * Overlap Matrix

In [17]:
## theoretical overlap matrix
OE = np.reshape(an['E'],(R,1))@np.reshape(am['E'],(1,R))
zetaOE = np.reshape(sigman['E'],(R,1))@np.reshape(sigmam['E'],(1,R))
deltaOE=zetaOE*(sigmavuE)
OE = deltaOE+OE
OE[0,:],OE[1,:]=OE[0,:]*sigv[0],OE[1,:]*sigv[1]
# OE[0,:],OE[1,:]=OE[0,:]*sigv[0],OE[1,:]*sigv[1]
OI = np.reshape(an['I'],(R,1))@np.reshape(am['I'],(1,R))
zetaOI = np.reshape(sigman['I'],(R,1))@np.reshape(sigmam['I'],(1,R))
deltaOI=zetaOI*(sigmavuI)
OI = deltaOI+OI
OI[0,:],OI[1,:]=OI[0,:]*sigv[0],OI[1,:]*sigv[1]



In [18]:
# testing the overlap matrix, numerical and theoretical
numOE=nout['E'].T@minp['E']
print('overlap matrix E, numerical:',numOE)
print('theoretical:',OE*NE)

numOI=nout['I'].T@minp['I']
print('overlap matrix I, numerical:',numOI)
print('theoretical:',OI*NI)

overlap matrix E, numerical: [[ 0.49106556  0.84036195]
 [ 0.38833521 -0.27360081]]
theoretical: [[ 0.47177591  0.86394181]
 [ 0.40063954 -0.26909061]]
overlap matrix I, numerical: [[0.24100978 0.31036699]
 [0.3182606  0.12582248]]
theoretical: [[0.2688     0.31988363]
 [0.31771424 0.10266642]]


## * Rotation

In [51]:
# after rotation
Orot=np.zeros((R,R))
Otheta = np.pi/2.0
Orot=np.array([[np.cos(Otheta),-np.sin(Otheta)],[np.sin(Otheta),np.cos(Otheta)]])
rotminp,rotnout={},{}
for ipop,iname in enumerate('EI'):
    rotminp[iname],rotnout[iname]=[],[]
    rotminp[iname],rotnout[iname]=minp[iname]@Orot,nout[iname]@Orot
    

In [57]:
# after rotation, we can also calculate the theoretical overlap matrix
# also compare with the numerical one
OE_=OE.copy()
OE_[0,0]=OE[0,0]*np.cos(Otheta)**2+OE[0,1]*np.cos(Otheta)*np.sin(Otheta)+OE[1,0]*np.cos(Otheta)*np.sin(Otheta)+OE[1,1]*np.sin(Otheta)**2
OE_[0,1]=-OE[0,0]*np.cos(Otheta)*np.sin(Otheta)+OE[0,1]*np.cos(Otheta)**2-OE[1,0]*np.sin(Otheta)**2+OE[1,1]*np.cos(Otheta)*np.sin(Otheta)
OE_[1,0]=-OE[0,0]*np.cos(Otheta)*np.sin(Otheta)-OE[0,1]*np.sin(Otheta)**2+OE[1,0]*np.cos(Otheta)**2+OE[1,1]*np.cos(Otheta)*np.sin(Otheta)
OE_[1,1]=OE[0,0]*np.sin(Otheta)**2-OE[0,1]*np.cos(Otheta)*np.sin(Otheta)-OE[1,0]*np.cos(Otheta)*np.sin(Otheta)+OE[1,1]*np.cos(Otheta)**2

OI_=OI.copy()
OI_[0,0]=OI[0,0]*np.cos(Otheta)**2+OI[0,1]*np.cos(Otheta)*np.sin(Otheta)+OI[1,0]*np.cos(Otheta)*np.sin(Otheta)+OI[1,1]*np.sin(Otheta)**2
OI_[0,1]=-OI[0,0]*np.cos(Otheta)*np.sin(Otheta)+OI[0,1]*np.cos(Otheta)**2-OI[1,0]*np.sin(Otheta)**2+OI[1,1]*np.cos(Otheta)*np.sin(Otheta)
OI_[1,0]=-OI[0,0]*np.cos(Otheta)*np.sin(Otheta)-OI[0,1]*np.sin(Otheta)**2+OI[1,0]*np.cos(Otheta)**2+OI[1,1]*np.cos(Otheta)*np.sin(Otheta)
OI_[1,1]=OI[0,0]*np.sin(Otheta)**2-OI[0,1]*np.cos(Otheta)*np.sin(Otheta)-OI[1,0]*np.cos(Otheta)*np.sin(Otheta)+OI[1,1]*np.cos(Otheta)**2


In [58]:
# compare theoretical and numerical overlap matrices, after rotation 
print('E overlap matrix after rotation')
print('numerical:',rotnout['E'].T@rotminp['E'])
print('theoretical:',OE_*NE)

print('I overlap matrix after rotation')
print('numerical:',rotnout['I'].T@rotminp['I'])
print('theoretical:',OI_*NI)

E overlap matrix after rotation
numerical: [[ 0.71938482 -0.16037249]
 [-0.57707188 -0.51871962]]
theoretical: [[ 0.73363333 -0.13878212]
 [-0.60208439 -0.53094803]]
I overlap matrix after rotation
numerical: [[ 0.49857014 -0.08373764]
 [-0.07580735 -0.12371565]]
theoretical: [[ 0.50453214 -0.0819821 ]
 [-0.08415148 -0.13306572]]


## * Figure Plots

In [26]:
## fontdicts and labels
fonttitle = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 14,
        }
ticksscatter = np.linspace(-0.06,0.06,3)
limscatter= [-0.061,0.061]

In [52]:
minp,nout=rotminp.copy(),rotnout.copy()
fig,ax=plt.subplots(figsize=(6,5))
colorlist='rb'
for ipop,iname in enumerate('EI'): #2 populations
    ax.scatter(minp[iname][:,0],nout[iname][:,0],s=2.0,c=colorlist[ipop],alpha=0.15)

ax.set_xlim(limscatter)
ax.set_ylim(limscatter)
ax.set_xticks(ticksscatter)
ax.set_yticks(ticksscatter)
ax.set_xlabel(r'$m_i^{(1),Q}$')#,fontdict=fonttitle)
ax.set_ylabel(r'$n_i^{(1),Q}$')#,fontdict=fonttitle)
ax.set_aspect('equal')
ax.grid()

fig,ax=plt.subplots(figsize=(6,5))
colorlist='rb'
for ipop,iname in enumerate('EI'): #2 populations
    ax.scatter(minp[iname][:,0],nout[iname][:,1],s=2.0,c=colorlist[ipop],alpha=0.15)

ax.set_xlim(limscatter)
ax.set_ylim(limscatter)
ax.set_xticks(ticksscatter)
ax.set_yticks(ticksscatter)
ax.set_xlabel(r'$m_i^{(1),Q}$')#,fontdict=fonttitle)
ax.set_ylabel(r'$n_i^{(2),Q}$')#,fontdict=fonttitle)
ax.set_aspect('equal')
ax.grid()

fig,ax=plt.subplots(figsize=(6,5))
colorlist='rb'
for ipop,iname in enumerate('EI'): #2 populations
    ax.scatter(minp[iname][:,1],nout[iname][:,0],s=2.0,c=colorlist[ipop],alpha=0.15)

ax.set_xlim(limscatter)
ax.set_ylim(limscatter)
ax.set_xticks(ticksscatter)
ax.set_yticks(ticksscatter)
ax.set_xlabel(r'$m_i^{(2),Q}$')#,fontdict=fonttitle)
ax.set_ylabel(r'$n_i^{(1),Q}$')#,fontdict=fonttitle)
ax.set_aspect('equal')
ax.grid()

fig,ax=plt.subplots(figsize=(6,5))
colorlist='rb'
for ipop,iname in enumerate('EI'): #2 populations
    ax.scatter(minp[iname][:,1],nout[iname][:,1],s=2.0,c=colorlist[ipop],alpha=0.15)

ax.set_xlim(limscatter)
ax.set_ylim(limscatter)
ax.set_xticks(ticksscatter)
ax.set_yticks(ticksscatter)
ax.set_xlabel(r'$m_i^{(2),Q}$')#,fontdict=fonttitle)
ax.set_ylabel(r'$n_i^{(2),Q}$')#,fontdict=fonttitle)
ax.set_aspect('equal')
ax.grid()

In [27]:
fig,ax=plt.subplots(2,2,figsize=(7,7))
colorlist='bg'
for r in range(R):
    for ipop,iname in enumerate('EI'): #2 populations
        ax[ipop][r].scatter(minp[iname][:,r],nout[iname][:,r],s=2.0,c=colorlist[0],alpha=0.15)
        # ax[ipop][r].scatter(rotminp[iname][:,r],rotnout[iname][:,r],s=2.0,c=colorlist[1],alpha=0.15)   
        # modify the figures
        ax[ipop][r].set_title(iname+' population, rank-'+str(r+1),fontdict=fonttitle)
        ax[ipop][r].set_xlim(limscatter)
        ax[ipop][r].set_ylim(limscatter)
        ax[ipop][r].set_xticks(ticksscatter)
        ax[ipop][r].set_yticks(ticksscatter)
        ax[ipop][r].set_aspect('equal')
        ax[ipop][r].grid()


In [24]:
fig,ax=plt.subplots(2,2,figsize=(7,7))
colorlist='rb'
# before the rotation
iop =0
for r in range(R):
    for ipop,iname in enumerate('EI'): #2 populations
        ax[iop][r].scatter(minp[iname][:,r],nout[iname][:,r],s=2.0,c=colorlist[ipop],alpha=0.15)
        # modify the figures
        ax[iop][r].set_title('before rotation, rank-'+str(r+1),fontdict=fonttitle)
        ax[iop][r].set_xlim(limscatter)
        ax[iop][r].set_ylim(limscatter)
        ax[iop][r].set_xticks(ticksscatter)
        ax[iop][r].set_yticks(ticksscatter)
        ax[iop][r].set_aspect('equal')
        ax[iop][r].grid()
# after rotation
iop+=1
for r in range(R):
    for ipop,iname in enumerate('EI'): #2 populations
        ax[iop][r].scatter(rotminp[iname][:,r],rotnout[iname][:,r],s=2.0,c=colorlist[ipop],alpha=0.15)   
        # modify the figures
        ax[iop][r].set_title('after rotation, rank-'+str(r+1),fontdict=fonttitle)
        ax[iop][r].set_xlim(limscatter)
        ax[iop][r].set_ylim(limscatter)
        ax[iop][r].set_xticks(ticksscatter)
        ax[iop][r].set_yticks(ticksscatter)
        ax[iop][r].set_aspect('equal')
        ax[iop][r].grid()

# Sparse Connectivity
* **simplest, E/I share the same sparseness**
* **check $\lambda,\sigma$**
* **different sparseness**

## E/I share the same sparseness $\epsilon$

## From the Simplest Rank-1 Case

In [2]:
import random
# simplest rank-1 case
JE,JI,a,b,g=0.9,1.9,0.0,0.0,0.9#2.9,1.5,0.3,0.2#
NE,NI=1000,1000
N = NE+NI
P=np.zeros((N,N))
P[:,:NE],P[:,NE:]=JE/NE,-JI/NI
J2=np.zeros((2,2))
# 2-by-2
J2[:,0],J2[:,1]=JE,-JI
lvec2,svr2,rvech2=la.svd(J2)

nit=40
randoms = np.linspace(0.01,0.99,nit)#np.linspace(0.5,0.9,20)
theou,theov,numu,numv=np.zeros((2,nit)),np.zeros((2,nit)),np.zeros((2,nit)),np.zeros((2,nit))# EI,RANK,ITERATIN
randnumu,randnumv=np.zeros((2,NE,nit)),np.zeros((2,NE,nit))
theosv,numsv = np.zeros(nit),np.zeros(nit)
rationum,stduratio,stdvratio,avguratio,avgvratio=np.zeros(nit),np.zeros((2,nit)),np.zeros((2,nit)),np.zeros((2,nit)),np.zeros((2,nit))

for idx,randomc in enumerate(randoms):
    sparseepsE,sparseepsI=(1-randomc),(1-randomc)
    Jdense=P.copy()
    Pmod=P.copy()
    denseu,densesv,densevh=la.svd(Jdense)
    densev=densevh.T
    theou[:,idx],theov[:,idx]=np.squeeze(denseu[1::NE,:1]),np.squeeze(densevh[:1,1::NE])
    theosv[idx]=densesv[:1]
    # integer
    for i in range(NE):
        selectE,selectI=int(NE*sparseepsE),int(NI*sparseepsI)
        sprsEidx,sprsIidx = random.sample(list(np.arange(NE)),k=selectE),random.sample(list(np.arange(NI)),k=selectI)
        sprsEidx,sprsIidx = np.array(sprsEidx),np.array(sprsIidx)
        sprsIidx=sprsIidx+NE
        Pmod[i,sprsEidx],Pmod[i,sprsIidx]=0,0
    for i in range(NE,NE+NI):
        selectE,selectI=int(NE*sparseepsE),int(NI*sparseepsI)
        sprsEidx,sprsIidx = random.sample(list(np.arange(NE)),k=selectE),random.sample(list(np.arange(NI)),k=selectI)
        sprsEidx,sprsIidx = np.array(sprsEidx),np.array(sprsIidx)
        sprsIidx=sprsIidx+NE
        Pmod[i,sprsEidx],Pmod[i,sprsIidx]=0,0
    Jsparse=Pmod.copy()
    # test left and right singular vectors
    sparseu,sparsesv,sparsevh=la.svd(Jsparse)
    sparsev =sparsevh.T
    numu[0,idx],numv[0,idx]=np.mean(sparseu[:NE,:1]),np.mean(sparsevh[:1,:NE].T)
    randnumu[0,:,idx],randnumv[0,:,idx]=np.squeeze(sparseu[:NE,:1]-numu[0,idx]),np.squeeze(sparsev[:NE,:1]-numv[0,idx])
    numu[1,idx],numv[1,idx]=np.mean(sparseu[NE:,:1]),np.mean(sparsevh[:1,NE:].T)
    randnumu[1,:,idx],randnumv[1,:,idx]=np.squeeze(sparseu[NE:,:1]-numu[1,idx]),np.squeeze(sparsev[NE:,:1]-numv[1,idx])
    numsv[idx]=sparsesv[:1]
    # quantitative
    rationum[idx]=numsv[idx]/theosv[idx]
    # print('ratio:',np.std(sparsev[NE:,0])*np.sqrt(NI)/np.mean(sparsev[NE:,0]))
    # print('ratio:',np.std(sparsev[:NE,0])*np.sqrt(NE)/np.mean(sparsev[:NE,0]))
    stduratio[0,idx],stduratio[1,idx]=np.std(sparseu[:NE,0])/denseu[0,0],np.std(sparseu[NE:,0])/denseu[-1,0]
    stdvratio[0,idx],stdvratio[1,idx]=np.std(sparsev[:NE,0])/densev[0,0],np.std(sparsev[NE:,0])/densev[-1,0]

    avguratio[0,idx],avguratio[1,idx]=np.mean(sparseu[:NE,0])/denseu[0,0],np.mean(sparseu[NE:,0])/denseu[-1,0]
    avgvratio[0,idx],avgvratio[1,idx]=np.mean(sparsev[:NE,0])/densev[0,0],np.mean(sparsev[NE:,0])/densev[-1,0]



* **sparsity determines the rescaling factor of singular values**
* **left singular vector $u$ barely has noise components, and right singular vector $v$ has noise components, furthermore $\mathrm{std}$ is proportional to corresponding mean value**

In [4]:
xs = 1/randoms
y = (stdvratio[0,:])**2
z = np.polyfit(xs,y,1)
print('polynomial:',z)
p = np.poly1d(z)
yest=p(xs)
fig,ax=plt.subplots(1,2,figsize=(5,5))
colorlist='br'
ax[0].plot(xs,y,colorlist[0])
ax[1].plot(randoms,y,colorlist[0])
ax[1].plot(randoms,yest,colorlist[1])
print(xs[0],xs[-1])
print('coefficient:',1/(NE+NI))

polynomial: [ 0.00049108 -0.00046123]
100.0 1.0101010101010102
coefficient: 0.0005


In [131]:
xs = 1/randoms
y = (stdvratio[0,:])**2
z = np.polyfit(xs,y,1)
print('polynomial:',z)
p = np.poly1d(z)
yest=p(xs)
fig,ax=plt.subplots(1,2,figsize=(5,5))
colorlist='br'
ax[0].plot(xs,y,colorlist[0])
ax[1].plot(randoms,y,colorlist[0])
ax[1].plot(randoms,yest,colorlist[1])
print(xs[0],xs[-1])
print('coefficient:',1/(NE+NI))

polynomial: [ 0.00048575 -0.00048301]
1.9607843137254901 1.0101010101010102
coefficient: 0.0005


In [134]:
## fontdicts and labels
fonttitle = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 14,
        }
tickslinear = np.linspace(0.0,1.0,2)
limslinear= [-0.01,1.01]

tickspartial = np.linspace(-0.5,1.0,4)
limspartial= [-0.51,1.1]

In [135]:
# also plot the ratio between sparse and dense matrix
fig,ax=plt.subplots(1,2,figsize=(10,8))
colorlist='br'
ax[0].plot(randoms,numsv[:]/theosv[:],colorlist[0])
ax[0].set_xlim(limslinear)
ax[0].set_ylim(limslinear)
ax[0].set_xticks(tickslinear)
ax[0].set_yticks(tickslinear)
ax[0].set_xlabel(r'$\epsilon$',fontsize=12)
ax[0].set_ylabel(r'$\tilde{\sigma}/ \sigma$',fontsize=12)
ax[0].set_aspect('equal')
ax[0].grid()

# plot how std ratio change with random ratio
# Plotting
# v
ax[1].scatter(randoms,stdvratio[0,:],s=8.0,c='r',marker='^',alpha=1.0)
ax[1].scatter(randoms,stdvratio[1,:],s=8.0,c='b',marker='^',alpha=1.0)
# u
ax[1].scatter(randoms,stduratio[0,:],s=8.0,c='y',marker='^',alpha=0.25)
ax[1].scatter(randoms,stduratio[1,:],s=8.0,c='g',marker='^',alpha=0.25)

# v average
ax[1].scatter(randoms,avgvratio[0,:],s=8.0,c='r',marker='*',alpha=1.0)
ax[1].scatter(randoms,avgvratio[1,:],s=8.0,c='b',marker='*',alpha=1.0)
# u average
ax[1].scatter(randoms,avguratio[0,:],s=8.0,c='y',marker='*',alpha=0.25)
ax[1].scatter(randoms,avguratio[1,:],s=8.0,c='g',marker='*',alpha=0.25)

ax[1].plot(randoms,stdvratio[0,:]**2+avgvratio[0,:]**2,linestyle='--',color=colorlist[1])

ax[1].set_xlim(limslinear)
ax[1].set_ylim(limspartial)
ax[1].set_xticks(tickslinear)
ax[1].set_yticks(tickspartial)
ax[1].set_xlabel(r'$\epsilon$',fontsize=12)
ax[1].set_ylabel('coefficients '+r'$\bar{v}, \zeta_v$',fontsize=12)
ax[1].set_aspect('equal')
ax[1].grid()

print('slope:',(stdvratio[0,-1]-stdvratio[0,0])/(randoms[-1]-randoms[0]))
print('v value:',numv[:,:2],numv[1,0]/numv[0,0])
print('vratio:',numv[:,:5]/theov[:,:5])

slope: 0.23149791277891624
v value: [[-0.01315985 -0.01344334]
 [ 0.0277819   0.02838038]] -2.1111111111111107
vratio: [[0.97211895 0.99306047 0.99578507 0.99734654 0.99782101]
 [0.97211895 0.99306047 0.99578507 0.99734654 0.99782101]]


In [136]:
ax[0].set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+'\n rescaling effect',fontdict=fonttitle)
ax[1].set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+'\n average v.s. noise components',fontdict=fonttitle)



Text(0.5, 1.0, '$J_E,\\ J_I=$0.9,1.9\n average v.s. noise components')

In [139]:
# also plot the ratio between sparse and dense matrix
fig,ax=plt.subplots(1,2,figsize=(7,7))
colorlist='br'
for r in range(1):
    ax[0].plot(randoms,stdvratio[0,:]**2+avgvratio[0,:]**2,colorlist[r+1])
    ax[1].scatter(randnumv[0,:,int(nit/2)],randnumv[1,:,int(nit/2)],s=5.0,c='g')#,marker='^',alpha=1.0)

In [ ]:
randomc=0.9
sparseepsE,sparseepsI=(1-randomc),(1-randomc)
Jdense=P.copy()
Pmod=P.copy()
denseu,densesv,densevh=la.svd(Jdense)
densev = densevh.T
print('original singular values:',densesv[:2])

# integer
for i in range(NE):
    selectE,selectI=int(NE*sparseepsE),int(NI*sparseepsI)
    sprsEidx,sprsIidx = random.sample(list(np.arange(NE)),k=selectE),random.sample(list(np.arange(NI)),k=selectI)
    sprsEidx,sprsIidx = np.array(sprsEidx),np.array(sprsIidx)
    sprsIidx=sprsIidx+NE
    Pmod[i,sprsEidx],Pmod[i,sprsIidx]=0,0
for i in range(NE,NE+NI):
    selectE,selectI=int(NE*sparseepsE),int(NI*sparseepsI)
    sprsEidx,sprsIidx = random.sample(list(np.arange(NE)),k=selectE),random.sample(list(np.arange(NI)),k=selectI)
    sprsEidx,sprsIidx = np.array(sprsEidx),np.array(sprsIidx)
    sprsIidx=sprsIidx+NE
    Pmod[i,sprsEidx],Pmod[i,sprsIidx]=0,0
    Jsparse=Pmod.copy()
    # test left and right singular vectors
sparseu,sparsesv,sparsevh=la.svd(Jsparse)
sparsev=sparsevh.T
print('sparse singular values:',sparsesv[:2])
print('ratio:',sparsesv[0]/densesv[0])


# Plotting
fig,ax=plt.subplots(figsize=(7,7))
ax.scatter(sparseu[:,0],sparsevh[0,:],s=2.0,c='b',alpha=0.15)
ax.scatter(denseu[:,0],densevh[0,:],s=2.0,c='r',alpha=1.0)
# ax.set_aspect('equal')
ax.grid()


In [90]:
print('denseu',denseu[:1,0],denseu[NE:NE+1,0])
print('sparseu',np.mean(sparseu[:NE,0]),np.mean(sparseu[NE:,0]))
print('std sparseu',np.std(sparseu[:NE,0])*np.sqrt(NE),np.std(sparseu[NE:,0])*np.sqrt(NI))

print('densev',densev[:1,0],densev[NE:NE+1,0])
print('sparsev',np.mean(sparsev[:NE,0]),np.mean(sparsev[NE:,0]))
print('std sparsev',np.std(sparsev[:NE,0])*np.sqrt(NE),np.std(sparsev[NE:,0])*np.sqrt(NI))
print('ratio:',np.std(sparsev[NE:,0])*np.sqrt(NI)/np.mean(sparsev[NE:,0]))
print('ratio:',np.std(sparsev[:NE,0])*np.sqrt(NE)/np.mean(sparsev[:NE,0]))

denseu [-0.02886751] [-0.02886751]
sparseu -0.028867523348143086 -0.02886750320038908
std sparseu 8.225271387266538e-05 7.78874523950362e-05
densev [-0.01516196] [0.0379049]
sparsev -0.015161216361712882 0.03790304090428213
std sparsev 0.003502240400577566 0.009270355827248434
ratio: 0.24458079367983132
ratio: -0.23099996181189583


In [88]:
print('denseu',denseu[:1,0],denseu[NE:NE+1,0])
print('sparseu',np.mean(sparseu[:NE,0]),np.mean(sparseu[NE:,0]))
print('std sparseu',np.std(sparseu[:NE,0])*np.sqrt(NE),np.std(sparseu[NE:,0])*np.sqrt(NI))

print('densev',densev[:1,0],densev[NE:NE+1,0])
print('sparsev',np.mean(sparsev[:NE,0]),np.mean(sparsev[NE:,0]))
print('std sparsev',np.std(sparsev[:NE,0])*np.sqrt(NE),np.std(sparsev[NE:,0])*np.sqrt(NI))
print('ratio:',np.std(sparsev[NE:,0])*np.sqrt(NI)/np.mean(sparsev[NE:,0]))
print('ratio:',np.std(sparsev[:NE,0])*np.sqrt(NE)/np.mean(sparsev[:NE,0]))

denseu [-0.02886751] [-0.02886751]
sparseu -0.028867346284504596 -0.028867678783810764
std sparseu 0.00017920024957300467 0.00017878032104974912
densev [-0.01516196] [0.0379049]
sparsev -0.015160399009607447 0.03790099752401851
std sparsev 0.005223472331177635 0.013368936723173675
ratio: 0.35273310985288847
ratio: -0.3445471539282981


In [86]:
print('denseu',denseu[:1,0],denseu[NE:NE+1,0])
print('sparseu',np.mean(sparseu[:NE,0]),np.mean(sparseu[NE:,0]))
print('std sparseu',np.std(sparseu[:NE,0])*np.sqrt(NE),np.std(sparseu[NE:,0])*np.sqrt(NI))

print('densev',densev[:1,0],densev[NE:NE+1,0])
print('sparsev',np.mean(sparsev[:NE,0]),np.mean(sparsev[NE:,0]))
print('std sparsev',np.std(sparsev[:NE,0])*np.sqrt(NE),np.std(sparsev[NE:,0])*np.sqrt(NI))
print('ratio:',np.std(sparsev[NE:,0])*np.sqrt(NI)/np.mean(sparsev[NE:,0]))
print('ratio:',np.std(sparsev[:NE,0])*np.sqrt(NE)/np.mean(sparsev[:NE,0]))

denseu [-0.02886751] [-0.02886751]
sparseu -0.02886757407236257 -0.028867437795459583
std sparseu 0.0005389503314029669 0.00048053480547151774
densev [-0.01516196] [0.0379049]
sparsev -0.015157461046091801 0.037893652615229444
std sparsev 0.009000159817439363 0.022637950233520433
ratio: 0.5974074461331355


In [45]:
JE,JI,a,b,g=1.10,1.0,0.6,-0.9,0.9#2.9,1.5,0.3,0.2#
NE,NI=600,600
N = NE+NI
P=np.zeros((N,N))
P[:,:NE],P[:,NE:]=JE/NE,-JI/NI
P[:NE,:NE],P[NE:,:NE]=P[:NE,:NE]+a/NE,P[NE:,:NE]-a/NE
P[:NE,NE:],P[NE:,NE:]=P[:NE,NE:]+b/NI,P[NE:,NE:]-b/NI
J2=np.zeros((2,2))
# 2-by-2
J2[:,0],J2[:,1]=JE,-JI
J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
lvec2,svr2,rvech2=la.svd(J2)
print('original singular values:',svr2)
cRatio=svr2[1]/svr2[0]

original singular values: [2.58239594 0.30204509]


In [53]:
import random
])*np.sqrt(NE),np.std(sparseu[NE:,0])*np.sqrt(NI),np.std(sparseu[:NE,1])*np.sqrt(NE),np.std(sparseu[NE:,1])*np.sqrt(NI))
# print('std div',np.std(sparseu[:NE,0])*np.sqrt(NE)/np.std(sparseu[NE:,0])/np.sqrt(NI),np.std(sparseu[:NE,1])*np.sqrt(NE)/np.std(sparseu[NE:,1])/np.sqrt(NI))
# print('ratio',1/randomc,'; svr2:',svr2)
# print('compare the singular values,')
# print('dense rank2:',densesv[:2])
# print('sparse rank2:',sparsesv[:2],(sparsesv[:2]/densesv[:2]))
sparseepsE,sparseepsI=(1-randomc),(1-randomc)
Jdense=P.copy()
Pmod=P.copy()
denseu,densesv,densevh=la.svd(Jdense)
# integer
for i in range(NE):
    selectE,selectI=int(NE*sparseepsE),int(NI*sparseepsI)
    sprsEidx,sprsIidx = random.sample(list(np.arange(NE)),k=selectE),random.sample(list(np.arange(NI)),k=selectI)
    sprsEidx,sprsIidx = np.array(sprsEidx),np.array(sprsIidx)
    sprsIidx=sprsIidx+NE
    Pmod[i,sprsEidx],Pmod[i,sprsIidx]=0,0
for i in range(NE,NE+NI):
    selectE,selectI=int(NE*sparseepsE),int(NI*sparseepsI)
    sprsEidx,sprsIidx = random.sample(list(np.arange(NE)),k=selectE),random.sample(list(np.arange(NI)),k=selectI)
    sprsEidx,sprsIidx = np.array(sprsEidx),np.array(sprsIidx)
    sprsIidx=sprsIidx+NE
    Pmod[i,sprsEidx],Pmod[i,sprsIidx]=0,0
    Jsparse=Pmod.copy()
    # test left and right singular vectors
sparseu,sparsesv,sparsevh=la.svd(Jsparse)



In [46]:
import random
randoms = np.linspace(0.1,0.5,20)#np.linspace(0.5,0.9,20)
theou,theov,numu,numv=np.zeros((2,2,20)),np.zeros((2,2,20)),np.zeros((2,2,20)),np.zeros((2,2,20))# EI,RANK,ITERATIN
theosv,numsv = np.zeros((2,20)),np.zeros((2,20))
for idx,randomc in enumerate(randoms):
    sparseepsE,sparseepsI=(1-randomc),(1-randomc)
    Jdense=P.copy()
    Pmod=P.copy()
    denseu,densesv,densevh=la.svd(Jdense)
    theou[:,:,idx],theov[:,:,idx]=denseu[1::NE,:2],densevh[:2,1::NE].T
    theosv[:,idx]=densesv[:2]
    # integer
    for i in range(NE):
        selectE,selectI=int(NE*sparseepsE),int(NI*sparseepsI)
        sprsEidx,sprsIidx = random.sample(list(np.arange(NE)),k=selectE),random.sample(list(np.arange(NI)),k=selectI)
        sprsEidx,sprsIidx = np.array(sprsEidx),np.array(sprsIidx)
        sprsIidx=sprsIidx+NE
        Pmod[i,sprsEidx],Pmod[i,sprsIidx]=0,0
    for i in range(NE,NE+NI):
        selectE,selectI=int(NE*sparseepsE),int(NI*sparseepsI)
        sprsEidx,sprsIidx = random.sample(list(np.arange(NE)),k=selectE),random.sample(list(np.arange(NI)),k=selectI)
        sprsEidx,sprsIidx = np.array(sprsEidx),np.array(sprsIidx)
        sprsIidx=sprsIidx+NE
        Pmod[i,sprsEidx],Pmod[i,sprsIidx]=0,0
    Jsparse=Pmod.copy()
    # test left and right singular vectors
    sparseu,sparsesv,sparsevh=la.svd(Jsparse)
    numu[0,:,idx],numv[0,:,idx]=np.mean(sparseu[:NE,:2]),np.mean(sparsevh[:2,:NE].T)
    numu[1,:,idx],numv[1,:,idx]=np.mean(sparseu[NE:,:2]),np.mean(sparsevh[:2,NE:].T)
    numsv[:,idx]=sparsesv[:2]



In [47]:
# fig,ax=plt.subplots(figsize=(7,7))
colorlist='br'
for r in range(2):
    ax.plot(randoms,numsv[r,:]/theosv[r,:],colorlist[r])


In [62]:
# print('compare the left singular vectors, deterministic:')
print('denseu',denseu[:1,0],denseu[NE:NE+1,0],denseu[:1,1],denseu[NE:NE+1,1])
print('sparseu',np.mean(sparseu[:NE,0]),np.mean(sparseu[NE:,0]),np.mean(sparseu[:NE,1]),np.mean(sparseu[NE:,1]))
unpertdense = np.zeros((2,2))
unpertdense[0,0],unpertdense[0,1],unpertdense[1,0],unpertdense[1,1]=denseu[:1,0],denseu[:1,1],denseu[NE:NE+1,0],denseu[NE:NE+1,1]
meansparse=np.zeros((2,2))
meansparse[0,0],meansparse[0,1],meansparse[1,0],meansparse[1,1]=np.mean(sparseu[:NE,0]),np.mean(sparseu[:NE,1]),np.mean(sparseu[NE:,0]),np.mean(sparseu[NE:,1])
divdensesparse=(meansparse)**2*NE
## CALCULATE THE OVERLAP
# densprsuE = denseu[:NE,:2].T@sparseu[:NE,:2]
# baseE = denseu[:NE,:2].T@denseu[:NE,:2]
# print('E dense+sparse:',densprsuE)#,baseE)

# densprsuI = denseu[NE:,:2].T@sparseu[NE:,:2]
# baseI = denseu[NE:,:2].T@denseu[NE:,:2]
# print('I dense+sparse:',densprsuI)#,baseI)
# print('sum',densprsuI+densprsuE)
# print('left vec:')
# print(sparseu[:,:2].T@sparseu[:,:2])

noiseampE=(np.std(sparseu[:NE,1])*np.sqrt(NE))**2
determampE=(np.mean(sparseu[:NE,1])**2)*NE
orgampE=np.sum(denseu[:NE,1]**2)
print('SUMM',noiseampE,determampE,';org:',orgampE)


print('std sparseu',np.std(sparseu[:NE,0])*np.sqrt(NE),np.std(sparseu[NE:,0])*np.sqrt(NI),np.std(sparseu[:NE,1])*np.sqrt(NE),np.std(sparseu[NE:,1])*np.sqrt(NI))
stdsparce
print('devid:',divdensesparse)
# print('std div',np.std(sparseu[:NE,0])*np.sqrt(NE)/np.std(sparseu[NE:,0])/np.sqrt(NI),np.std(sparseu[:NE,1])*np.sqrt(NE)/np.std(sparseu[NE:,1])/np.sqrt(NI))
# print('ratio',1/randomc,'; svr2:',svr2)
# print('compare the singular values,')
# print('dense rank2:',densesv[:2])
# print('sparse rank2:',sparsesv[:2],(sparsesv[:2]/densesv[:2]))

denseu [-0.04029768] [-0.00653941] [-0.00653941] [0.04029768]
sparseu -0.04029872865930424 -0.006532847453999447 0.006531640835423543 -0.04029340727278192
SUMM 0.00025345687071516045 0.025597399201783413 ;org: 0.025658350974743414
std sparseu 0.0007770760343585988 0.00014037365771638014 0.015920328850722917 0.0037339171570838353
devid: [[0.97439252 0.0255974 ]
 [0.02560686 0.9741352 ]]


In [356]:
print('ratio:',np.sqrt(baseE[1,1]/densprsuE[1,1])+1))
print('ratio:',np.sqrt(-baseI[1,1]/densprsuI[1,1]-1))

print('ratio:',np.sqrt(1/(baseI[0,0]/densprsuI[0,0])))
print('ratio:',np.sqrt(-baseE[0,0]/densprsuE[0,0]+1))

ratio: 0.09002161813663881
ratio: 0.055601579305443495
ratio: 0.9979891823427893
ratio: 0.011729947472719835


In [56]:
fig,ax=plt.subplots(1,2,figsize=(7,7))
for r in range(2):
    ax[r].scatter(sparseu[:,r],sparsevh[r,:],s=2.0,c='b',alpha=0.15)
    ax[r].scatter(denseu[:,r],densevh[r,:],s=2.0,c='r',alpha=1.0)
    # modify the figures
    # ax[r].set_title('before rotation, rank-'+str(r+1),fontdict=fonttitle)
    # ax[r].set_xlim(limscatter)
    # ax[r].set_ylim(limscatter)
    # ax[r].set_xticks(ticksscatter)
    # ax[r].set_yticks(ticksscatter)
    ax[r].set_aspect('equal')
    ax[r].grid()

In [102]:
at=np.reshape(morg[0,:],(1,2))
the = np.pi/4.0
rotmat=np.array([[np.cos(the),-np.sin(the)],[np.sin(the),np.cos(the)]])
atr=at@rotmat#mrot[0,:]#
# atr=rotmat@at.T
# atr =atr.T
plt.plot([0,at[0,0]],[0,at[0,1]],'r')
plt.plot([0,atr[0,0]],[0,atr[0,1]],'orange')
plt.grid()
# plt('square')
# plt.axis(square)

# General Rotation and Shape 10/12/2020
* case 1 zero-mean distribution for E and I populations
* case 2 unitary standard deviation but different mean (should be easy, only exist rotation)
* case 3 different mean as well as different $\sigma$

In [197]:
# case 1, zero-mean and different sigma, to see how things change with rotation
# mean values for m and n
class LowrankNN(object):
    def __init__(self,Netsettings,celltypenum,pmE,pmI,pnE,pnI,pnbase,P=['E','I'],Rotation=np.pi/4.0):
        self.populations = P
        self.rank = Netsettings['rank']
        if P[0]=='E' and P[1]=='I':
            NE,NI=celltypenum['E'],celltypenum['I']
        # declare the rank structure (vectors)
        R=Netsettings['rank']
        self.m,self.n = np.zeros((NE+NI,R)),np.zeros((NE+NI,R))
        self.mrot,self.nrot=np.zeros((NE+NI,R)),np.zeros((NE+NI,R))
        self.Enoise,self.Inoise=np.zeros((2*R,2*R)),np.zeros((2*R,2*R))
        # declare to be outputted 
        self.overlapmatE,self.overlapmatE=np.zeros((2*R,2*R)),np.zeros((2*R,2*R))
        self.overlapmatrotE,self.overlapmatrotI=np.zeros((2*R,2*R)),np.zeros((2*R,2*R))
        self.overlapEnoise,self.overlapInoise=np.zeros((2*R,2*R)),np.zeros((2*R,2*R))
        self.rotation=Rotation
        self.pformE,self.pformI,self.pfornE,self.pfornI,self.pnbase=pmE,pmI,pnE,pnI,pnbase
    def matRotation(self,rotationchange):
        self.rotmat=np.zeros((self.rank,self.rank))
        if rotationchange=='default':
            the=self.rotation#np.pi/4.0#
        else:
            the=rotationchange
        self.rotmat=np.array([[np.cos(the),-np.sin(the)],[np.sin(the),np.cos(the)]])
        
        self.mrot=self.m@self.rotmat
        self.nrot=self.n@self.rotmat
        print(self.mrot[:,1].T@self.nrot[:,0])
    def Gaussian_distribution(self,celltypenum,mu_m,sig_m,mu_n,sig_n):
        # the mean values
        # for m and n mu
        NE,NI= celltypenum['E'],celltypenum['I']
        N=NE+NI
        R = self.rank
        pformE,pformI,pfornE,pfornI=self.pformE,self.pformI,self.pfornE,self.pfornI
        # for m and n sigma
        # generate iid distribution 
        m22=np.random.randn(N,R)
        # random rotation
        thetan=self.pnbase*np.pi
        Othetan=np.array([[np.cos(thetan),-np.sin(thetan)],[np.sin(thetan),np.cos(thetan)]])
        n22=Othetan@m22.T
        n22=n22.T  
        # generate distribution for E
        for i in range(R): # rank
            # while (1):
            #     p1,p2=np.random.random(2)
            #     if p1**2+p2**2<1.0:
            #         p3=np.sqrt(1-p1**2-p2**2)
            #         break
            self.m[:NE,i]=pformE[i,0]*m22[:NE,0]+pformE[i,1]*m22[:NE,1]+pformE[i,2]*np.random.randn(NE)
        # record the overlap for mE  
        self.overlapEnoise[0,0],self.overlapEnoise[0,1],self.overlapEnoise[1,1]=np.sum(self.m[:NE,0]*self.m[:NE,0]),np.sum(self.m[:NE,0]*self.m[:NE,1]),np.sum(self.m[:NE,1]*self.m[:NE,1])
        self.overlapEnoise[1,0]=self.overlapEnoise[0,1]

        # generate distribution for I
        for i in range(R): # rank
            # while (1):
            #     p1,p2=np.random.random(2)
            #     if p1**2+p2**2<1.0:
            #         p3=np.sqrt(1-p1**2-p2**2)
            #         break
            self.m[NE:,i]=pformI[i,0]*m22[NE:,0]+pformI[i,1]*m22[NE:,1]+pformI[i,2]*np.random.randn(NI)
        # record the overlap for mE  
        self.overlapInoise[0,0],self.overlapInoise[0,1],self.overlapInoise[1,1]=np.sum(self.m[NE:,0]*self.m[NE:,0]),np.sum(self.m[NE:,0]*self.m[NE:,1]),np.sum(self.m[NE:,1]*self.m[NE:,1])
        self.overlapInoise[1,0]=self.overlapInoise[0,1]

        # FOR N
        # generate distribution for E
        for i in range(R): # rank
            # while (1):
            #     p1,p2=np.random.random(2)
            #     if p1**2+p2**2<1.0:
            #         p3=np.sqrt(1-p1**2-p2**2)
            #         break
            self.n[:NE,i]=pfornE[i,0]*n22[:NE,0]+pfornE[i,1]*n22[:NE,1]+pfornE[i,2]*np.random.randn(NE)
        # record the overlap for mE  
        self.overlapEnoise[R+0,R+0],self.overlapEnoise[R+0,R+1],self.overlapEnoise[R+1,R+1]=np.sum(self.n[:NE,0]*self.n[:NE,0]),np.sum(self.n[:NE,0]*self.n[:NE,1]),np.sum(self.n[:NE,1]*self.n[:NE,1])
        self.overlapEnoise[R+1,R+0]=self.overlapEnoise[R+0,R+1]
        # record the overlap for mEnE
        self.overlapEnoise[0,R+0],self.overlapEnoise[0,R+1],self.overlapEnoise[1,R+1]=np.sum(self.m[:NE,0]*self.n[:NE,0]),np.sum(self.m[:NE,0]*self.n[:NE,1]),np.sum(self.m[:NE,1]*self.n[:NE,1])
        self.overlapEnoise[1,R+0]=np.sum(self.m[:NE,1]*self.n[:NE,0])
        # record the overlap for mE  
        self.overlapEnoise[R+0,0],self.overlapEnoise[R+0,1],self.overlapEnoise[R+1,1]=np.sum(self.n[:NE,0]*self.m[:NE,0]),np.sum(self.n[:NE,0]*self.m[:NE,1]),np.sum(self.n[:NE,1]*self.m[:NE,1])
        self.overlapEnoise[R+1,0]=np.sum(self.n[:NE,1]*self.m[:NE,0])

        # generate distribution for I
        for i in range(R): # rank
            # while (1):
            #     p1,p2=np.random.random(2)
            #     if p1**2+p2**2<1.0:
            #         p3=np.sqrt(1-p1**2-p2**2)
            #         break
            self.n[NE:,i]=pfornI[i,0]*n22[NE:,0]+pfornI[i,1]*n22[NE:,1]+pfornI[i,2]*np.random.randn(NI)
        # record the overlap for mE  
        self.overlapInoise[R+0,R+0],self.overlapInoise[R+0,R+1],self.overlapInoise[R+1,R+1]=np.sum(self.n[NE: ,0]*self.n[NE:,0]),np.sum(self.n[NE:,0]*self.n[NE:,1]),np.sum(self.n[NE:,1]*self.n[NE:,1])
        self.overlapInoise[R+1,R+0]=self.overlapInoise[R+0,R+1]
        # record the overlap for mEnE
        self.overlapInoise[0,R+0],self.overlapInoise[0,R+1],self.overlapInoise[1,R+1]=np.sum(self.m[NE:,0]*self.n[NE:,0]),np.sum(self.m[NE:,0]*self.n[NE:,1]),np.sum(self.m[NE:,1]*self.n[NE:,1])
        self.overlapInoise[1,R+0]=np.sum(self.m[NE:,1]*self.n[NE:,0])
        # record the overlap for mE  
        self.overlapInoise[R+0,0],self.overlapInoise[R+0,1],self.overlapInoise[R+1,1]=np.sum(self.n[NE:,0]*self.m[NE:,0]),np.sum(self.n[NE:,0]*self.m[NE:,1]),np.sum(self.n[NE:,1]*self.m[NE:,1])
        self.overlapInoise[R+1,0]=np.sum(self.n[NE:,1]*self.m[NE:,0])

        for i in range (self.rank):
            self.m[:NE,i]*=sig_m['E'][i]
            self.m[NE:,i]*=sig_m['I'][i]
            self.n[:NE,i]*=sig_n['E'][i]
            self.n[NE:,i]*=sig_n['I'][i]

        for i in range (self.rank):
            self.m[:NE,i]+=mu_m['E'][i]
            self.m[NE:,i]+=mu_m['I'][i]
            self.n[:NE,i]+=mu_n['E'][i]
            self.n[NE:,i]+=mu_n['I'][i]

        


In [108]:
print(pformE,pformI,pfornE,pfornI,pnbase)

[[0.51795368 0.70430166 0.48547209]
 [0.14116051 0.98792886 0.06379869]] [[0.5315635  0.09922291 0.8411867 ]
 [0.12339738 0.74804008 0.65208061]] [[0.64100615 0.76196903 0.09227304]
 [0.22036893 0.92564014 0.30761643]] [[0.08792906 0.64712832 0.75729348]
 [0.55946689 0.19279484 0.80611845]] 0.18958359276804826


In [231]:
R,nn,P=2,1000,2#P1=E,P2=I
Netsettings={'rank':2,
}
celltypenum={'E':2000,
            'I':2000,
}
# mu_m={'E':np.array([0.0,0.0]),
#       'I':np.array([0.0,0.0]),
# }
# mu_n={'E':np.array([0.0,0.0]),
#       'I':np.array([0.0,0.0]),
# }
mu_m={'E':np.array([0.30,0.10]),
      'I':np.array([-0.10,-0.20]),
}
mu_n={'E':np.array([-0.40,0.20]),
      'I':np.array([0.10,-0.30]),
}

# sig_m={'E':np.array([0.0,0.0]),
#       'I':np.array([0.0,0.0]),
# }
# sig_n={'E':np.array([0.0,0.0]),
#       'I':np.array([0.0,0.0]),
# }

sig_m={'E':np.array([0.9,0.5]),
      'I':np.array([0.64,1.3]),
}
sig_n={'E':np.array([1.6,0.23]),
      'I':np.array([0.54,0.95]),
}

# # two overlap
# # generate distribution for E
# pformE=np.zeros((2,3))
# for i in range(R): # rank
#       while (1):
#             p1,p2=np.random.random(2)
#             if p1**2+p2**2<1.0:
#                   p3=np.sqrt(1-p1**2-p2**2)
#                   break
#       pformE[i,:]=np.array([p1,p2,p3])
# # generate disturbution for I
# pformI=np.zeros((2,3))
# for i in range(R): # rank
#       while (1):
#             p1,p2=np.random.random(2)
#             if p1**2+p2**2<1.0:
#                   p3=np.sqrt(1-p1**2-p2**2)
#                   break
#       pformI[i,:]=np.array([p1,p2,p3])

# pfornE=np.zeros((2,3))
# for i in range(R): # rank
#       while (1):
#             p1,p2=np.random.random(2)
#             if p1**2+p2**2<1.0:
#                   p3=np.sqrt(1-p1**2-p2**2)
#                   break
#       pfornE[i,:]=np.array([p1,p2,p3])
# # generate disturbution for I
# pfornI=np.zeros((2,3))
# for i in range(R): # rank
#       while (1):
#             p1,p2=np.random.random(2)
#             if p1**2+p2**2<1.0:
#                   p3=np.sqrt(1-p1**2-p2**2)
#                   break
#       pfornI[i,:]=np.array([p1,p2,p3])
# pnbase=np.random.random()

In [232]:
theta=np.pi/6.956118754525706
GeneralRotation=LowrankNN(Netsettings,celltypenum,pformE,pformI,pfornE,pfornI,pnbase,P=['E','I'],Rotation=np.pi/2.0)
GeneralRotation.Gaussian_distribution(celltypenum,mu_m,sig_m,mu_n,sig_n)
morg,norg=GeneralRotation.m,GeneralRotation.n
GeneralRotation.matRotation('default')
mrot,nrot=GeneralRotation.mrot,GeneralRotation.nrot
NE=celltypenum['E']

-874.2757482869563


In [233]:
ticks = np.linspace(-1.0,1.0,3)
lims= [-1.1,1.1]

ticks = np.linspace(-5.0,5.0,3)
lims= [-5.1,5.1]

## E and I Gaussian distributions

In [234]:
# before rotation
fig,ax=plt.subplots(4,4,figsize=(9,9))
for i in range(2):
    for j in range(2):
        ax[i][j].scatter(morg[:NE,i],morg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[i][j].scatter(morg[NE:,i],morg[NE:,j],s=5.0,c='blue',alpha=0.05)
for i in range(2):
    for j in range(2):
        ax[2+i][2+j].scatter(norg[:NE,i],norg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[2+i][2+j].scatter(norg[NE:,i],norg[NE:,j],s=5.0,c='blue',alpha=0.05)
        
for i in range(2):
    for j in range(2):
        ax[i][2+j].scatter(morg[:NE,i],norg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[i][2+j].scatter(morg[NE:,i],norg[NE:,j],s=5.0,c='blue',alpha=0.05)
for i in range(2):
    for j in range(2):
        ax[2+i][j].scatter(norg[:NE,i],morg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[2+i][j].scatter(norg[NE:,i],morg[NE:,j],s=5.0,c='blue',alpha=0.05)
for i in range(4):
    for j in range(4):
        ax[i][j].set_xlim(lims)
        ax[i][j].set_ylim(lims)
        ax[i][j].set_xticks(ticks)
        ax[i][j].set_yticks(ticks)
        ax[i][j].set_aspect('equal')
        ax[i][j].grid()
# for i in range(4):
#     for j in range(4):
#         ax[i][j].set_xlim(lims)
#         ax[i][j].set_ylim(lims)
#         ax[i][j].set_xticks([])#ticks)
#         ax[i][j].set_yticks([])#ticks)
#         ax[i][j].set_aspect('equal')
#         ax[i][j].grid()
ax[0][3].set_xticks(ticks)
ax[0][3].set_yticks(ticks)
ax[3][0].set_xticks(ticks)
ax[3][0].set_yticks(ticks)
ax[0][1].set_xticks(ticks)
ax[0][1].set_yticks(ticks)
ax[1][0].set_xticks(ticks)
ax[1][0].set_yticks(ticks)
# ax[0].scatter(morg[NE:,0],morg[NE:,1],c='green',alpha=0.5)

## how to rotate

In [ ]:
def quadratic(a,b,c):
    m= b*b-4*a*c
    if m>=0:
        x = (-b+math.sqrt(m))/(2*a)
        y = (-b-math.sqrt(m))/(2*a)
        return x,y
    else:
        return'No Answer!'

In [216]:
PureOVE=morg[:NE,:].T@norg[:NE,:]
a,b,c=PureOVE[0,1],PureOVE[0,0]-PureOVE[1,1],-PureOVE[1,0]
tantheta=quadratic(a,b,c)
thetarot=np.arctan(tantheta[1])
GeneralRotation.matRotation(thetarot)
mrotE,nrotE=GeneralRotation.mrot,GeneralRotation.nrot

# # ROT I
# PureOVI=morg[NE:,:].T@norg[NE:,:]
# a,b,c=PureOVI[0,1],PureOVI[0,0]-PureOVI[1,1],-PureOVI[1,0]
# tantheta=quadratic(a,b,c)
# thetarot=np.arctan(tantheta[0])
# GeneralRotation.matRotation(thetarot)
# mrotI,nrotI=GeneralRotation.mrot,GeneralRotation.nrot



-243.13519962690955


In [82]:
# 0,3 figure
vec2dE,vec2dI=np.zeros((2,NE)),np.zeros((2,NE))
vec2dE[0,:],vec2dE[1,:]=morg[:NE,0],norg[:NE,1]
vec2dI[0,:],vec2dI[1,:]=morg[NE:,0],norg[NE:,1]
mu2dE,mu2dI=np.zeros((2,1)),np.zeros((2,1))
mu2dE[0,0],mu2dE[1,0]=mu_m['E'][0],mu_n['E'][1]
mu2dI[0,0],mu2dI[1,0]=mu_m['I'][0],mu_n['I'][1]
noise2dE,noise2dI=vec2dE.copy(),vec2dI.copy()
for i in range(2):
    noise2dE[i,:]-=mu2dE[i,0]
    noise2dI[i,:]-=mu2dI[i,0]
corr22E,corr22I=noise2dE@noise2dE.T,noise2dI@noise2dI.T
eigvE,eigvecE=la.eig(corr22E)
eigvI,eigvecI=la.eig(corr22I)
for i in range(2):
    ax[0][3].plot([mu2dE[0,0],mu2dE[0,0]+6*eigvecE[0,i]],[mu2dE[1,0],mu2dE[1,0]+6*eigvecE[1,i]],'r--',linewidth=1.0)
    ax[0][3].plot([mu2dI[0,0],mu2dI[0,0]+6*eigvecI[0,i]],[mu2dI[1,0],mu2dI[1,0]+6*eigvecI[1,i]],'b--',linewidth=1.0)

for i in range(2):
    ax[3][0].plot([mu2dE[1,0],mu2dE[1,0]+6*eigvecE[1,i]],[mu2dE[0,0],mu2dE[0,0]+6*eigvecE[0,i]],'r--',linewidth=1.0)
    ax[3][0].plot([mu2dI[1,0],mu2dI[1,0]+6*eigvecI[1,i]],[mu2dI[0,0],mu2dI[0,0]+6*eigvecI[0,i]],'b--',linewidth=1.0)



## E rotation

In [235]:
# mrot,nrot=mrotE,nrotE
# before rotation
fig,ax=plt.subplots(4,4,figsize=(9,9))
for i in range(2):
    for j in range(2):
        ax[i][j].scatter(morg[:NE,i],morg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[i][j].scatter(mrot[:NE,i],mrot[:NE,j],s=5.0,c='orange',alpha=0.05)
for i in range(2):
    for j in range(2):
        ax[2+i][2+j].scatter(norg[:NE,i],norg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[2+i][2+j].scatter(nrot[:NE,i],nrot[:NE,j],s=5.0,c='orange',alpha=0.05)
        
for i in range(2):
    for j in range(2):
        ax[i][2+j].scatter(morg[:NE,i],norg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[i][2+j].scatter(mrot[:NE,i],nrot[:NE,j],s=5.0,c='orange',alpha=0.05)
for i in range(2):
    for j in range(2):
        ax[2+i][j].scatter(norg[:NE,i],morg[:NE,j],s=5.0,c='red',alpha=0.05)
        ax[2+i][j].scatter(nrot[:NE,i],mrot[:NE,j],s=5.0,c='orange',alpha=0.05)
for i in range(4):
    for j in range(4):
        ax[i][j].set_xlim(lims)
        ax[i][j].set_ylim(lims)
        ax[i][j].set_xticks(ticks)
        ax[i][j].set_yticks(ticks)
        ax[i][j].set_aspect('equal')
        ax[i][j].grid()
# for i in range(4):
#     for j in range(4):
#         ax[i][j].set_xlim(lims)
#         ax[i][j].set_ylim(lims)
#         ax[i][j].set_xticks([])#ticks)
#         ax[i][j].set_yticks([])#ticks)
#         ax[i][j].set_aspect('equal')
#         ax[i][j].grid()
ax[0][3].set_xticks(ticks)
ax[0][3].set_yticks(ticks)
ax[0][1].set_xticks(ticks)
ax[0][1].set_yticks(ticks)
ax[2][3].set_xticks(ticks)
ax[2][3].set_yticks(ticks)

In [236]:
# rotate 
vec2dE,vec2dI=np.zeros((2,NE)),np.zeros((2,NE))
vec2dE[0,:],vec2dE[1,:]=norg[:NE,0],norg[:NE,1]
vec2dI[0,:],vec2dI[1,:]=nrot[:NE,0],nrot[:NE,1]
mu2dE,mu2dI=np.zeros((2,1)),np.zeros((2,1))
mu2dE[0,0],mu2dE[1,0]=mu_n['E'][0],mu_n['E'][1]
mu2dI[0,0],mu2dI[1,0]=np.mean(vec2dI[0,:]),np.mean(vec2dI[1,:])
noise2dE,noise2dI=vec2dE.copy(),vec2dI.copy()
for i in range(2):
    noise2dE[i,:]-=mu2dE[i,0]
    noise2dI[i,:]-=mu2dI[i,0]
corr22E,corr22I=noise2dE@noise2dE.T,noise2dI@noise2dI.T
eigvE,eigvecE=la.eig(corr22E)
eigvI,eigvecI=la.eig(corr22I)
for i in range(2):
    ax[2][3].plot([mu2dE[0,0],mu2dE[0,0]+6*eigvecE[0,i]],[mu2dE[1,0],mu2dE[1,0]+6*eigvecE[1,i]],'r',linewidth=1.0,linestyle='--')
    ax[2][3].plot([mu2dI[0,0],mu2dI[0,0]+6*eigvecI[0,i]],[mu2dI[1,0],mu2dI[1,0]+6*eigvecI[1,i]],'orange',linewidth=1.0,linestyle='--')

for i in range(2):
    ax[3][2].plot([mu2dE[1,0],mu2dE[1,0]+6*eigvecE[1,i]],[mu2dE[0,0],mu2dE[0,0]+6*eigvecE[0,i]],'red',linewidth=1.0,linestyle='--')
    ax[3][2].plot([mu2dI[1,0],mu2dI[1,0]+6*eigvecI[1,i]],[mu2dI[0,0],mu2dI[0,0]+6*eigvecI[0,i]],'orange',linewidth=1.0,linestyle='--')

vec2dE,vec2dI=np.zeros((2,NE)),np.zeros((2,NE))
vec2dE[0,:],vec2dE[1,:]=morg[:NE,0],morg[:NE,1]
vec2dI[0,:],vec2dI[1,:]=mrot[:NE,0],mrot[:NE,1]
mu2dE,mu2dI=np.zeros((2,1)),np.zeros((2,1))
mu2dE[0,0],mu2dE[1,0]=mu_m['E'][0],mu_m['E'][1]
mu2dI[0,0],mu2dI[1,0]=np.mean(vec2dI[0,:]),np.mean(vec2dI[1,:])
noise2dE,noise2dI=vec2dE.copy(),vec2dI.copy()
for i in range(2):
    noise2dE[i,:]-=mu2dE[i,0]
    noise2dI[i,:]-=mu2dI[i,0]
corr22E,corr22I=noise2dE@noise2dE.T,noise2dI@noise2dI.T
eigvE,eigvecE=la.eig(corr22E)
eigvI,eigvecI=la.eig(corr22I)
for i in range(2):
    ax[0][1].plot([mu2dE[0,0],mu2dE[0,0]+6*eigvecE[0,i]],[mu2dE[1,0],mu2dE[1,0]+6*eigvecE[1,i]],'red',linewidth=1.0,linestyle='--')
    ax[0][1].plot([mu2dI[0,0],mu2dI[0,0]+6*eigvecI[0,i]],[mu2dI[1,0],mu2dI[1,0]+6*eigvecI[1,i]],'orange',linewidth=1.0,linestyle='--')


# not rotate
for i in range(2):
    for j in range(2):
        vec2dE,vec2dI=np.zeros((2,NE)),np.zeros((2,NE))
        vec2dE[0,:],vec2dE[1,:]=morg[:NE,i],norg[:NE,j]
        vec2dI[0,:],vec2dI[1,:]=mrot[:NE,i],nrot[:NE,j]
        mu2dE,mu2dI=np.zeros((2,1)),np.zeros((2,1))
        mu2dE[0,0],mu2dE[1,0]=mu_m['E'][0],mu_n['E'][1]
        mu2dI[0,0],mu2dI[1,0]=np.mean(vec2dI[0,:]),np.mean(vec2dI[1,:])
        noise2dE,noise2dI=vec2dE.copy(),vec2dI.copy()
        for k in range(2):
            noise2dE[i,:]-=mu2dE[k,0]
            noise2dI[i,:]-=mu2dI[k,0]
        corr22E,corr22I=noise2dE@noise2dE.T,noise2dI@noise2dI.T
        eigvE,eigvecE=la.eig(corr22E)
        eigvI,eigvecI=la.eig(corr22I)
        for k in range(2):
            ax[i][j+2].plot([mu2dE[0,0],mu2dE[0,0]+6*eigvecE[0,k]],[mu2dE[1,0],mu2dE[1,0]+6*eigvecE[1,k]],'r',linewidth=1.0,linestyle='--')
            ax[i][j+2].plot([mu2dI[0,0],mu2dI[0,0]+6*eigvecI[0,k]],[mu2dI[1,0],mu2dI[1,0]+6*eigvecI[1,k]],'orange',linewidth=1.0,linestyle='--')


## I rotation

In [237]:
# before rotation
fig,ax=plt.subplots(4,4,figsize=(9,9))
for i in range(2):
    for j in range(2):
        ax[i][j].scatter(morg[NE:,i],morg[NE:,j],s=5.0,c='blue',alpha=0.05)
        ax[i][j].scatter(mrot[NE:,i],mrot[NE:,j],s=5.0,c='green',alpha=0.05)
for i in range(2):
    for j in range(2):
        ax[2+i][2+j].scatter(norg[NE:,i],norg[NE:,j],s=5.0,c='blue',alpha=0.05)
        ax[2+i][2+j].scatter(nrot[NE:,i],nrot[NE:,j],s=5.0,c='green',alpha=0.05)
        
for i in range(2):
    for j in range(2):
        ax[i][2+j].scatter(morg[NE:,i],norg[NE:,j],s=5.0,c='blue',alpha=0.05)
        ax[i][2+j].scatter(mrot[NE:,i],nrot[NE:,j],s=5.0,c='green',alpha=0.05)
for i in range(2):
    for j in range(2):
        ax[2+i][j].scatter(norg[NE:,i],morg[NE:,j],s=5.0,c='blue',alpha=0.05)
        ax[2+i][j].scatter(nrot[NE:,i],mrot[NE:,j],s=5.0,c='green',alpha=0.05)
for i in range(4):
    for j in range(4):
        ax[i][j].set_xlim(lims)
        ax[i][j].set_ylim(lims)
        ax[i][j].set_xticks(ticks)
        ax[i][j].set_yticks(ticks)
        ax[i][j].set_aspect('equal')
        ax[i][j].grid()
# for i in range(4):
#     for j in range(4):
#         ax[i][j].set_xlim(lims)
#         ax[i][j].set_ylim(lims)
#         ax[i][j].set_xticks([])#ticks)
#         ax[i][j].set_yticks([])#ticks)
#         ax[i][j].set_aspect('equal')
#         ax[i][j].grid()
ax[0][3].set_xticks(ticks)
ax[0][3].set_yticks(ticks)
ax[3][0].set_xticks(ticks)
ax[3][0].set_yticks(ticks)



In [238]:
# rotate 
vec2dE,vec2dI=np.zeros((2,NE)),np.zeros((2,NE))
vec2dE[0,:],vec2dE[1,:]=norg[NE:,0],norg[NE:,1]
vec2dI[0,:],vec2dI[1,:]=nrot[NE:,0],nrot[NE:,1]
mu2dE,mu2dI=np.zeros((2,1)),np.zeros((2,1))
mu2dE[0,0],mu2dE[1,0]=mu_n['I'][0],mu_n['I'][1]
mu2dI[0,0],mu2dI[1,0]=np.mean(vec2dI[0,:]),np.mean(vec2dI[1,:])
noise2dE,noise2dI=vec2dE.copy(),vec2dI.copy()
for i in range(2):
    noise2dE[i,:]-=mu2dE[i,0]
    noise2dI[i,:]-=mu2dI[i,0]
corr22E,corr22I=noise2dE@noise2dE.T,noise2dI@noise2dI.T
eigvE,eigvecE=la.eig(corr22E)
eigvI,eigvecI=la.eig(corr22I)
for i in range(2):
    ax[2][3].plot([mu2dE[0,0],mu2dE[0,0]+6*eigvecE[0,i]],[mu2dE[1,0],mu2dE[1,0]+6*eigvecE[1,i]],'blue',linewidth=1.0,linestyle='--')
    ax[2][3].plot([mu2dI[0,0],mu2dI[0,0]+6*eigvecI[0,i]],[mu2dI[1,0],mu2dI[1,0]+6*eigvecI[1,i]],'green',linewidth=1.0,linestyle='--')

for i in range(2):
    ax[3][2].plot([mu2dE[1,0],mu2dE[1,0]+6*eigvecE[1,i]],[mu2dE[0,0],mu2dE[0,0]+6*eigvecE[0,i]],'blue',linewidth=1.0,linestyle='--')
    ax[3][2].plot([mu2dI[1,0],mu2dI[1,0]+6*eigvecI[1,i]],[mu2dI[0,0],mu2dI[0,0]+6*eigvecI[0,i]],'green',linewidth=1.0,linestyle='--')

vec2dE,vec2dI=np.zeros((2,NE)),np.zeros((2,NE))
vec2dE[0,:],vec2dE[1,:]=morg[NE:,0],morg[NE:,1]
vec2dI[0,:],vec2dI[1,:]=mrot[NE:,0],mrot[NE:,1]
mu2dE,mu2dI=np.zeros((2,1)),np.zeros((2,1))
mu2dE[0,0],mu2dE[1,0]=mu_m['I'][0],mu_m['I'][1]
mu2dI[0,0],mu2dI[1,0]=np.mean(vec2dI[0,:]),np.mean(vec2dI[1,:])
noise2dE,noise2dI=vec2dE.copy(),vec2dI.copy()
for i in range(2):
    noise2dE[i,:]-=mu2dE[i,0]
    noise2dI[i,:]-=mu2dI[i,0]
corr22E,corr22I=noise2dE@noise2dE.T,noise2dI@noise2dI.T
eigvE,eigvecE=la.eig(corr22E)
eigvI,eigvecI=la.eig(corr22I)
for i in range(2):
    ax[0][1].plot([mu2dE[0,0],mu2dE[0,0]+6*eigvecE[0,i]],[mu2dE[1,0],mu2dE[1,0]+6*eigvecE[1,i]],'blue',linewidth=1.0,linestyle='--')
    ax[0][1].plot([mu2dI[0,0],mu2dI[0,0]+6*eigvecI[0,i]],[mu2dI[1,0],mu2dI[1,0]+6*eigvecI[1,i]],'green',linewidth=1.0,linestyle='--')


# not rotate
for i in range(2):
    for j in range(2):
        vec2dE,vec2dI=np.zeros((2,NE)),np.zeros((2,NE))
        vec2dE[0,:],vec2dE[1,:]=morg[NE:,i],norg[NE:,j]
        vec2dI[0,:],vec2dI[1,:]=mrot[NE:,i],nrot[NE:,j]
        mu2dE,mu2dI=np.zeros((2,1)),np.zeros((2,1))
        mu2dE[0,0],mu2dE[1,0]=mu_m['I'][i],mu_n['I'][j]
        mu2dI[0,0],mu2dI[1,0]=np.mean(vec2dI[0,:]),np.mean(vec2dI[1,:])
        noise2dE,noise2dI=vec2dE.copy(),vec2dI.copy()
        for k in range(2):
            noise2dE[k,:]-=mu2dE[k,0]
            noise2dI[k,:]-=mu2dI[k,0]
        corr22E,corr22I=noise2dE@noise2dE.T,noise2dI@noise2dI.T
        eigvE,eigvecE=la.eig(corr22E)
        eigvI,eigvecI=la.eig(corr22I)
        for k in range(2):
            ax[i][j+2].plot([mu2dE[0,0],mu2dE[0,0]+6*eigvecE[0,k]],[mu2dE[1,0],mu2dE[1,0]+6*eigvecE[1,k]],'blue',linewidth=1.0,linestyle='--')
            ax[i][j+2].plot([mu2dI[0,0],mu2dI[0,0]+6*eigvecI[0,k]],[mu2dI[1,0],mu2dI[1,0]+6*eigvecI[1,k]],'green',linewidth=1.0,linestyle='--')

In [41]:
# overlap matrix 
print('E overlap:',GeneralRotation.overlapEnoise)
print('I overlap:',GeneralRotation.overlapInoise)
np.savez('12Decorg.npy',Eoverlap=GeneralRotation.overlapEnoise,Ioverlap=GeneralRotation.overlapInoise,morg=morg,norg=norg,mu_m=mu_m,mu_n=mu_n,sigma_m=sig_m,sigma_n=sig_n)


E overlap: [[1031.00895857  201.1840473    58.46436431  182.51692657]
 [ 201.1840473  1033.2511556   137.49548049  305.14839357]
 [  58.46436431  137.49548049 1038.45697692  155.15900681]
 [ 182.51692657  305.14839357  155.15900681 1034.5766521 ]]
I overlap: [[1042.51687381  715.43911748  796.63373925  305.21068435]
 [ 715.43911748 1065.75553685  621.89644664  364.4857629 ]
 [ 796.63373925  621.89644664 1003.40971724  344.29557907]
 [ 305.21068435  364.4857629   344.29557907 1000.93162863]]


In [8]:
fig,ax=plt.subplots(4,4,figsize=(9,9))
for i in range(2):
    for j in range(2):
        ax[i][j].scatter(morg[:NE,i],morg[:NE,j],s=5.0,c='red',alpha=0.5)
        ax[i][j].scatter(mrot[:NE,i],mrot[:NE,j],s=5.0,c='blue',alpha=0.5)
for i in range(2):
    for j in range(2):
        ax[2+i][2+j].scatter(norg[:NE,i],norg[:NE,j],s=5.0,c='red',alpha=0.5)
        ax[2+i][2+j].scatter(nrot[:NE,i],nrot[:NE,j],s=5.0,c='blue',alpha=0.5)

for i in range(2):
    for j in range(2):
        ax[i][2+j].scatter(morg[:NE,i],norg[:NE,j],s=5.0,c='red',alpha=0.5)
        ax[i][2+j].scatter(mrot[:NE,i],nrot[:NE,j],s=5.0,c='blue',alpha=0.5)
for i in range(2):
    for j in range(2):
        ax[2+i][j].scatter(norg[:NE,i],morg[:NE,j],s=5.0,c='red',alpha=0.5)
        ax[2+i][j].scatter(nrot[:NE,i],mrot[:NE,j],s=5.0,c='blue',alpha=0.5)
for i in range(4):
    for j in range(4):
        ax[i][j].set_xlim(lims)
        ax[i][j].set_ylim(lims)
        ax[i][j].set_xticks(ticks)
        ax[i][j].set_yticks(ticks)
        ax[i][j].set_aspect('equal')
        ax[i][j].grid()
# ax[0].scatter(morg[NE:,0],morg[NE:,1],c='green',alpha=0.5)

In [11]:
ticks = np.linspace(-4.0,4.0,5)
lims= [-4.1,4.1]

In [16]:
print(np.arctan2(1,-1)/np.pi*180)

135.0


In [27]:
# test angle and overlap matrix
nnn=6000
zetax,zetay,zetaz=np.random.randn(nnn),np.random.randn(nnn),np.random.randn(nnn)
rho,siga,sigb=0.5,0.8,0.4
ya=siga*(rho*zetax+np.sqrt(1-rho**2)*zetay)
xa=sigb*zetax
print('overlap:',siga*sigb*rho)
print('theo:',np.sum(ya*xa)/nnn)
mat=np.array([[siga**2,siga*rho*sigb],[siga*rho*sigb,sigb**2]])
eigv,eigvec=la.eig(mat)
theta=np.arctan2(eigvec[0,0],eigvec[1,0])
fig,ax=plt.subplots(figsize=(6,6))
ax.scatter(xa,ya,c='red',alpha=0.5)
ax.plot([0,4*np.cos(theta)],[0,4*np.sin(theta)],'b',linewidth=1.5)
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_aspect('equal')
ax.grid()
print()

overlap: 0.16000000000000003
theo: 0.15612573246182648



# Backup Code

In [ ]:
#  15112020 reconfinement
JE,JI=2.9,1.5
a,b=0.6,-0.9#0.3,0.2#
# calculate the rotation matrix in rank-2 case (reduced), then extend to large 
# generate perturbed matrix
J2=np.zeros((2,2))
J2[:,0],J2[:,1]=JE,-JI
J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
lvec2,svr2,rvech2=la.svd(J2)
M2 = np.array([[1,1],[1,-1]])
ROT = np.sqrt(2)*la.inv(M2)@lvec2 # M@ROT.T/sqrt(2?)=u
print('sv2:',svr2)

'''
Generate two expressions in large random matrix
params:
    nn = E/I neuron number
    M/Nrank = low rank structure in EI format
    U/V.    = low rank structyre in SVD format
    M/Nranktilde = low rank EI format for large random matrix
    U/Vtilde = low rank SVD format for large random matrix
    sigvmat: diagonal singular values matrix
    
0],s=10.0,c='red',marker='^')
# ax[0].scatter(Ubarnum[:nn,1],Ubarnum[nn:,1],s=10.0,c='red',marker='^')
# xnorm = 1.0/np.sqrt(nn)
# ax.set_xlim([-xnorm,xnorm])
# ax.set_ylim([-xnorm,xnorm])
# ax.set_aspect('equal')

'''
grandom=3.2
theta = grandom
Rc,R = 0,2
for i in range(2):
    if svr2[i]>grandom:
        Rc+=1
nn=600
sigvmat,rhosigv = np.zeros((R,R)),np.zeros((R,R))
for i in range(R):
    sigvmat[i,i]=svr2[i]
    rhosigv[i,i]=svr2[i]+(grandom**2)/svr2[i]
invsigvmat = la.inv(sigvmat)
# print('inverse singular value matrix:',invsigvmat)
# print('square of the inverse singular value matrix:',la.matrix_power(invsigvmat,2)) 

zetau,zetav = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
nx,ny = np.random.randn(nn*2,R)/1,np.random.randn(nn*2,R)/nn
zetau=nx.copy()
zetav=ny.copy()
zetaM,zetaN = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
# generate rank-2 matrix P without random entries
P=np.zeros((nn*2,nn*2))
P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
# use SVD to decompose the low rank matrix
lvec,svr,rvech=la.svd(P)
rvec=rvech.T 
# check the singular value
# print(svr,svr2)
if np.sum((svr[:2]-svr2[:])**2)>1e-8:  
    print('Singular values for 2by2 matrix do not match those for NbyN matrix!')

alphamat= np.eye(R)-(grandom**2)*la.matrix_power(invsigvmat,2)
Asqrt=np.eye(R)#alphamat.copy()
for i in range(R):
    if alphamat[i,i]>0.0:
        Asqrt[i,i]=np.sqrt(alphamat[i,i])
    else:
        Asqrt[i,i]=0.0
print('determined:',Asqrt)
randomterm=np.zeros((R,R))
for i in range(R):
    randomterm[i,i]=np.sqrt(1.0-Asqrt[i,i]**2)
    print('random:',randomterm[i,i])
# EI format
withrand,wrank = 1.0,1
M1,N1=np.ones(2*nn),np.ones(2*nn)
N1[:nn],N1[nn:]=JE/nn,-JI/nn
M2,N2=M1.copy(),N1.copy()
M2[:nn],M2[nn:]=1,-1
N2[:nn],N2[nn:]=a/nn,b/nn
Mrank,Nrank=np.zeros((nn*2,R)),np.zeros((nn*2,R))
NT = np.random.randn(nn*2,R)
NT = NT.T 
if R>1:
    Mrank[:,0],Mrank[:,1],Nrank[:,0],Nrank[:,1]=M1,M2,N1,N2
else:
    Mrank[:,0],Nrank[:,0]=M1,N1
Mranktilde = wrank*Mrank@ROT@Asqrt+withrand*zetaM@randomterm*np.sqrt(2*nn)
Nranktildeh = (sigvmat+(grandom**2)*invsigvmat)@(wrank*Asqrt@invsigvmat@ROT.T@Nrank.T+withrand*randomterm@zetaN.T/np.sqrt(2*nn))#+(sigvmat+invsigvmat)@invsigvmat@zetav.T
Nranktilde=Nranktildeh.T 

U,V=np.zeros_like(Mrank),np.zeros_like(Nrank)
U,V=lvec[:,:R],rvec[:,:R]
Utilde,Vtilde = wrank*U@Asqrt+withrand*zetau@randomterm,wrank*V@Asqrt+withrand*zetav@randomterm

# add random part in, Xtilde = X+P
X = (grandom)*np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
Xtilde=X+P
if Rc>1:
    Xtilde_svdrec = Utilde@(sigvmat+(grandom**2)*invsigvmat)@Vtilde.T#/grandom**2
    Xtilde_eirec  = Mranktilde@Nranktildeh#Nranktildeh@Mranktilde#/grandom**2
else:
    Xtilde_svdrec = np.reshape(Utilde[:,0],(2*nn,1))*(sigvmat[0,0]+(grandom**2)*invsigvmat[0,0])@np.reshape(Vtilde[:,0],(1,nn*2))#@#/grandom**2
    Xtilde_eirec  = np.reshape(Mranktilde[:,0],(2*nn,1))@np.reshape(Nranktilde[:,0],(1,2*nn))#np.reshape(Nranktildeh[0,:],(1,2*nn))@np.reshape(Mranktilde[:,0],(nn*2,1))#/grandom**2

# test two expressions, using singular values
_,sigvxtilde,_=la.svd(Xtilde)
_,sigveitilde,_=la.svd(Xtilde_eirec)
_,sigvsvdtilde,_=la.svd(Xtilde_svdrec)

print('compare the singular values from')
print('original:',sigvxtilde[:3])
print('svd:',sigvsvdtilde[:3])
print('ei:',sigveitilde[:3])#Xtilde_eirec)#




# Large Random Matrix Has Two Outliers
** here we set the g->for random matrix, smaller than the second singular value of the low-rank matrix, thus we have 2 outliers **

In [23]:
# SCATTER PLOT of Mtilde and Ntilde
bimeanMtilde,bimeanNtilde=np.zeros((R,Rc)),np.zeros((R,Rc))
# Mranktilde,Mrank,Nranktilde,Nrank=-Mranktilde/np.sqrt(2*nn),-Mrank/np.sqrt(2*nn),-Nranktilde*np.sqrt(2*nn),-Nrank*np.sqrt(2*nn)
for i in range(Rc):
    bimeanMtilde[0,i],bimeanNtilde[0,i]=np.mean(Mranktilde[:nn,i]),np.mean(Nranktilde[:nn,i])
    bimeanMtilde[1,i],bimeanNtilde[1,i]=np.mean(Mranktilde[nn:,i]),np.mean(Nranktilde[nn:,i])
RANKP=0
fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(111)
ax1.scatter(-Mranktilde[:,RANKP],-Nranktilde[:,RANKP],c='pink',alpha=0.5)
ax1.set_aspect('equal')
ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)
RANKP=1
fig=plt.figure(figsize=(6,6))
ax2=fig.add_subplot(111)
ax2.scatter(-Mranktilde[:,RANKP],-Nranktilde[:,RANKP],c='pink',alpha=0.5)
ax2.set_aspect('equal')
ax2.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax2.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

nVtilde = Vtilde@(sigvmat+(grandom**2)*invsigvmat)
nV = V@sigvmat
bimeanUtilde,bimeanVtilde=np.zeros((R,R)),np.zeros((R,R))
for i in range(Rc):
    bimeanUtilde[0,i],bimeanVtilde[0,i]=np.mean(Utilde[:nn,i]),np.mean(nVtilde[:nn,i])
    bimeanUtilde[1,i],bimeanVtilde[1,i]=np.mean(Utilde[nn:,i]),np.mean(nVtilde[nn:,i])
RANKP=0
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax1.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax1.set_aspect('equal')
ax1.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax1.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

RANKP=1
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax2.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax2.set_aspect('equal')
ax2.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax2.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

ax2.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax2.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)


In [25]:
ax1.set_xlabel(r'$\tilde{M}^{(1)}$',fontsize=14)
ax1.set_ylabel(r'$\tilde{N}^{(1)}$',fontsize=14)
ax1.set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+', $a, b=$'+str(a)+','+str(b)+'\n $\sigma_1=$'+str(format(svr2[0],'.1f'))+r',$\rho_1=$'+str(format(rhosigv[0,0],'.1f'))+',$\overline{\sigma}=$'+str(grandom),fontsize=16)

ax2.set_xlabel(r'$\tilde{M}^{(2)}$',fontsize=14)
ax2.set_ylabel(r'$\tilde{N}^{(2)}$',fontsize=14)
ax2.set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+', $a, b=$'+str(a)+','+str(b)+'\n $\sigma_2=$'+str(format(svr2[1],'.1f'))+r',$\rho_2=$'+str(format(rhosigv[1,1],'.1f'))+',$\overline{\sigma}=$'+str(grandom),fontsize=16)

Text(0.5, 1.0, '$J_E,\\ J_I=$2.9,1.5, $a, b=$0.6,-0.9\n $\\sigma_2=$0.7,$\\rho_2=$1.1,$\\overline{\\sigma}=$0.5')

In [5]:
# SCATTER PLOT of Mtilde and Ntilde
bimeanMtilde,bimeanNtilde=np.zeros((R,Rc)),np.zeros((R,Rc))
Mranktilde,Mrank,Nranktilde,Nrank=-Mranktilde/np.sqrt(2*nn),-Mrank/np.sqrt(2*nn),-Nranktilde*np.sqrt(2*nn),-Nrank*np.sqrt(2*nn)
for i in range(Rc):
    bimeanMtilde[0,i],bimeanNtilde[0,i]=np.mean(Mranktilde[:nn,i]),np.mean(Nranktilde[:nn,i])
    bimeanMtilde[1,i],bimeanNtilde[1,i]=np.mean(Mranktilde[nn:,i]),np.mean(Nranktilde[nn:,i])
RANKP=0
fig=plt.figure(figsize=(6,6))
ax1=fig.add_subplot(111)
ax1.scatter(-Mranktilde[:,RANKP],-Nranktilde[:,RANKP],c='pink',alpha=0.5)
ax1.set_aspect('auto')
ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)

nVtilde = Vtilde@(sigvmat+(grandom**2)*invsigvmat)
nV = V@sigvmat
bimeanUtilde,bimeanVtilde=np.zeros((R,R)),np.zeros((R,R))
for i in range(Rc):
    bimeanUtilde[0,i],bimeanVtilde[0,i]=np.mean(Utilde[:nn,i]),np.mean(nVtilde[:nn,i])
    bimeanUtilde[1,i],bimeanVtilde[1,i]=np.mean(Utilde[nn:,i]),np.mean(nVtilde[nn:,i])
RANKP=0
# fig=plt.figure(figsize=(5,5))
# ax1=fig.add_subplot(111)
ax1.scatter(Utilde[:,RANKP],nVtilde[:,RANKP],c='yellow',alpha=0.5)
ax1.set_aspect('auto')
ax1.scatter(U[:,RANKP],nV[:,RANKP],c='green',alpha=1.0)
ax1.scatter(bimeanUtilde[:,RANKP],bimeanVtilde[:,RANKP],c='red',alpha=0.8)

# ax1.scatter(-Mrank[:,RANKP],-Nrank[:,RANKP],c='b',alpha=1.0)
# ax1.scatter(-bimeanMtilde[:,RANKP],-bimeanNtilde[:,RANKP],c='red',alpha=1.0)



In [7]:
ax1.scatter(-Mrank[:,1],-Nrank[:,1],c='cyan',alpha=0.5)
ax1.set_xlabel(r'$\tilde{M}^{(1)}$',fontsize=14)
ax1.set_ylabel(r'$\tilde{N}^{(1)}$',fontsize=14)
ax1.set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+', $a, b=$'+str(a)+','+str(b),fontsize=16)

Text(0.5, 1.0, '$J_E,\\ J_I=$2.9,1.5, $a, b=$0.6,-0.9')

## Figures: Gaussian Distribution
* The figures with red and yellow and pink dots M v.s. N etc.etc

In [80]:
## Gaussian distributions for Utilde,U and Vtilde,V

# numeerically obtain Utildenum, Vtildenum
Utildenum,rho,Vtildenumh=la.svd(Xtilde)
Utildenum = Utildenum[:,:R]
Vtildenumh = Vtildenumh[:R,:]
Vtildenum = Vtildenumh.T 
Udiffnum,Vdiffnum=Utildenum-U@Asqrt,Vtildenum-V@Asqrt
Udiffnum[:,1]+=2*U[:,1]*Asqrt[1,1]
Vdiffnum[:,1]+=2*V[:,1]*Asqrt[1,1]

Nspace = 1000
# left singular matrix
maxstd = np.std(np.reshape(Udiffnum,(1,-1)))
fig=plt.figure()
ax1=fig.add_subplot(211)
ax2=fig.add_subplot(212)
n, bins, patches = ax1.hist(Udiffnum[:,0], 100, density=True, alpha=0.5)
stdtest=np.std(Udiffnum[:,0])
xxx=np.linspace(-3*stdtest,3*stdtest,Nspace)
stdu=1.0/(svr[0]*np.sqrt(2*nn))
disttest = np.exp(-(xxx/stdu)**2/2)/np.sqrt(2*np.pi*stdu**2)
ax1.plot(xxx,disttest*nn*2/Nspace,label='theoretical '+r'$\zeta$ in $\tilde{u}^{(1)}$')
ax1.set_xlim([-5*maxstd,5*maxstd])
ax1.legend()
n, bins, patches = ax2.hist(Udiffnum[:,1], 100, density=True, alpha=0.5)
stdtest=np.std(Udiffnum[:,1])
xxx=np.linspace(-3*stdtest,3*stdtest,Nspace)
stdu=1.0/(svr[1]*np.sqrt(2*nn))
disttest = np.exp(-(xxx/stdu)**2/2)/np.sqrt(2*np.pi*stdu**2)
# disttest/=np.sum(disttest)
ax2.plot(xxx,disttest*nn*2/Nspace,label='theoretical '+r'$\zeta$ in $\tilde{u}^{(2)}$')
ax2.set_xlim([-5*maxstd,5*maxstd])
ax2.set_xlabel('Entries')
ax2.legend()


In [86]:
# numeerically obtain Mtildenum, Ntildenum
Utildenum,rho,Vtildenumh=la.svd(Xtilde)
Utildenum = Utildenum[:,:R]
Vtildenumh = Vtildenumh[:R,:]
Vtildenum = Vtildenumh.T 
Mtilde_t=Utildenum-np.random.randn(nn*2,R)@la.matrix_power(invsigvmat,3)/np.sqrt(2*nn)
Mtildenum=Mtilde_t*np.sqrt(2*nn)@la.inv(Asqrt)
Mdiffnum =Mtildenum-Mrank@ROT
# Udiffnum[:,1]+=2*U[:,1]*Asqrt[1,1]
# Vdiffnum[:,1]+=2*V[:,1]*Asqrt[1,1]

Nspace = 1000
# left singular matrix
maxstd = np.std(np.reshape(Mdiffnum,(1,-1)))
fig=plt.figure()
ax1=fig.add_subplot(211)
ax2=fig.add_subplot(212)
n, bins, patches = ax1.hist(Mdiffnum[:,0], 100, density=True, alpha=0.5)
stdtest=np.std(Mdiffnum[:,0])
xxx=np.linspace(-3*stdtest,3*stdtest,Nspace)
stdu=1.0/(svr[0]*1)
disttest = np.exp(-(xxx/stdu)**2/2)/np.sqrt(2*np.pi*stdu**2)
ax1.plot(xxx,disttest*nn*2/Nspace,label='theoretical '+r'$\zeta$ in $\tilde{u}^{(1)}$')
ax1.set_xlim([-5*maxstd,5*maxstd])
ax1.legend()
n, bins, patches = ax2.hist(Mdiffnum[:,1], 100, density=True, alpha=0.5)
stdtest=np.std(Mdiffnum[:,1])
xxx=np.linspace(-3*stdtest,3*stdtest,Nspace)
stdu=1.0/(svr[1])
disttest = np.exp(-(xxx/stdu)**2/2)/np.sqrt(2*np.pi*stdu**2)
# disttest/=np.sum(disttest)
ax2.plot(xxx,disttest*nn*2/Nspace,label='theoretical '+r'$\zeta$ in $\tilde{u}^{(2)}$')
ax2.set_xlim([-5*maxstd,5*maxstd])
ax2.set_xlabel('Entries')
ax2.legend()

## Testing Two Representations
* the shadow area represents trial standard deviation (that figure)

In [14]:
#  15112020 reconfinement
JE,JI=1.0,1.45
a,b=0.40,0.50#0.3,0.2#
nvalues,niterations=20,50
aseries = np.linspace(0.0,1.0,nvalues)
# calculate the rotation matrix in rank-2 case (reduced), then extend to large 
# generate perturbed matrix
grandom=0.5
Singularvalue=2*grandom*np.ones((3,2,nvalues,niterations))
for iter in range(niterations):
    for idxa, a in enumerate(aseries):
        J2=np.zeros((2,2))
        J2[:,0],J2[:,1]=JE,-JI
        J2[0,0],J2[1,0]=J2[0,0]+a,J2[1,0]-a
        J2[0,1],J2[1,1]=J2[0,1]+b,J2[1,1]-b
        lvec2,svr2,rvech2=la.svd(J2)
        M2 = np.array([[1,1],[1,-1]])
        ROT = np.sqrt(2)*la.inv(M2)@lvec2 # M@ROT.T/sqrt(2?)=u
        # print('Rotation:',ROT)

        '''
        Generate two expressions in large random matrix
        params:
            nn = E/I neuron number
            M/Nrank = low rank structure in EI format
            U/V.    = low rank structyre in SVD format
            M/Nranktilde = low rank EI format for large random matrix
            U/Vtilde = low rank SVD format for large random matrix
            sigvmat: diagonal singular values matrix
            

        '''
        theta = grandom
        Rc,R = 0,2
        for i in range(2):
            if svr2[i]>grandom:
                Rc+=1
        nn=600
        sigvmat,rhosigv = np.zeros((R,R)),np.zeros((R,R))
        for i in range(R):
            sigvmat[i,i]=svr2[i]
            rhosigv[i,i]=svr2[i]+(grandom**2)/svr2[i]
        invsigvmat = la.inv(sigvmat)
        # print('inverse singular value matrix:',invsigvmat)
        # print('square of the inverse singular value matrix:',la.matrix_power(invsigvmat,2)) 

        zetau,zetav = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
        zetaM,zetaN = np.random.randn(nn*2,R)/np.sqrt(2*nn),np.random.randn(nn*2,R)/np.sqrt(2*nn)
        # generate rank-2 matrix P without random entries
        P=np.zeros((nn*2,nn*2))
        P[:,:nn],P[:,nn:]=JE/nn,-JI/nn
        P[:nn,:nn],P[nn:,:nn]=P[:nn,:nn]+a/nn,P[nn:,:nn]-a/nn
        P[:nn,nn:],P[nn:,nn:]=P[:nn,nn:]+b/nn,P[nn:,nn:]-b/nn
        # use SVD to decompose the low rank matrix
        lvec,svr,rvech=la.svd(P)
        rvec=rvech.T 
        # check the singular value
        # print(svr,svr2)
        if np.sum((svr[:2]-svr2[:])**2)>1e-8:  
            print('Singular values for 2by2 matrix do not match those for NbyN matrix!')

        alphamat= np.eye(R)-(grandom**2)*la.matrix_power(invsigvmat,2)
        Asqrt=np.eye(R)#alphamat.copy()
        for i in range(R):
            if alphamat[i,i]>0.0:
                Asqrt[i,i]=np.sqrt(alphamat[i,i])
            else:
                Asqrt[i,i]=0.0
        randomterm=np.zeros((R,R))
        for i in range(R):
            randomterm[i,i]=np.sqrt(1.0-Asqrt[i,i]**2)
        # EI format
        withrand,wrank = 1,1
        M1,N1=np.ones(2*nn),np.ones(2*nn)
        N1[:nn],N1[nn:]=JE/nn,-JI/nn
        M2,N2=M1.copy(),N1.copy()
        M2[:nn],M2[nn:]=1,-1
        N2[:nn],N2[nn:]=a/nn,b/nn
        Mrank,Nrank=np.zeros((nn*2,R)),np.zeros((nn*2,R))
        NT = np.random.randn(nn*2,R)
        NT = NT.T 
        if R>1:
            Mrank[:,0],Mrank[:,1],Nrank[:,0],Nrank[:,1]=M1,M2,N1,N2
        else:
            Mrank[:,0],Nrank[:,0]=M1,N1
        Mranktilde = wrank*Mrank@ROT@Asqrt+withrand*zetaM@randomterm*np.sqrt(2*nn)
        Nranktildeh = (sigvmat+(grandom**2)*invsigvmat)@(wrank*Asqrt@invsigvmat@ROT.T@Nrank.T+withrand*randomterm@zetaN.T/np.sqrt(2*nn))#+(sigvmat+invsigvmat)@invsigvmat@zetav.T
        Nranktilde=Nranktildeh.T 

        U,V=np.zeros_like(Mrank),np.zeros_like(Nrank)
        U,V=lvec[:,:R],rvec[:,:R]
        Utilde,Vtilde = wrank*U@Asqrt+withrand*zetau@randomterm,wrank*V@Asqrt+withrand*zetav@randomterm

        # add random part in, Xtilde = X+P
        X = (grandom)*np.random.randn(2*nn,2*nn)/np.sqrt(2*nn)
        Xtilde=X+P
        if Rc>1:
            Xtilde_svdrec = Utilde@(sigvmat+(grandom**2)*invsigvmat)@Vtilde.T#/grandom**2
            Xtilde_eirec  = Mranktilde@Nranktildeh#Nranktildeh@Mranktilde#/grandom**2
        else:
            Xtilde_svdrec = np.reshape(Utilde[:,0],(2*nn,1))*(sigvmat[0,0]+(grandom**2)*invsigvmat[0,0])@np.reshape(Vtilde[:,0],(1,nn*2))#@#/grandom**2
            Xtilde_eirec  = np.reshape(Mranktilde[:,0],(2*nn,1))@np.reshape(Nranktilde[:,0],(1,2*nn))#np.reshape(Nranktildeh[0,:],(1,2*nn))@np.reshape(Mranktilde[:,0],(nn*2,1))#/grandom**2
        
        # test two expressions, using singular values
        _,sigvxtilde,_=la.svd(Xtilde)
        _,sigveitilde,_=la.svd(Xtilde_eirec)
        _,sigvsvdtilde,_=la.svd(Xtilde_svdrec)

        if Rc<2:
            Singularvalue[:,0,idxa,iter]=np.array([sigvxtilde[0],sigveitilde[0],sigvsvdtilde[0]])
        else:
            Singularvalue[0,:,idxa,iter]=sigvxtilde[:Rc]
            Singularvalue[1,:,idxa,iter]=sigveitilde[:Rc]
            Singularvalue[2,:,idxa,iter]=sigvsvdtilde[:Rc]

np.save('Singularvalue18Nov_1.npy', Singularvalue)  
# print('compare the singular values from')
# print('original:',sigvxtilde[:2])
# print('svd:',sigvsvdtilde[:2])
# print('ei:',sigveitilde[:2])




In [322]:
data = np.load('Singularvalue17Nov.npy')
Singularvalues = data
print(np.shape(Singularvalues))
ntype,nsigv,nseries,niterations=np.shape(Singularvalues)
# calculate std
meanstdseries = np.zeros((ntype,nsigv,nseries,2))
for it in range(ntype):
    for isv in range(nsigv):
        for isr in range(nseries):
            meanstdseries[it,isv,isr,0]=np.mean(np.squeeze(Singularvalues[it,isv,isr,:]))
            meanstdseries[it,isv,isr,1]=np.std(np.squeeze(Singularvalues[it,isv,isr,:]))


(3, 2, 20, 50)


In [15]:
Singularvalues=Singularvalue.copy()
ntype,nsigv,nseries,niterations=np.shape(Singularvalues)
# calculate std
meanstdseries = np.zeros((ntype,nsigv,nseries,2))
for it in range(ntype):
    for isv in range(nsigv):
        for isr in range(nseries):
            meanstdseries[it,isv,isr,0]=np.mean(np.squeeze(Singularvalues[it,isv,isr,:]))
            meanstdseries[it,isv,isr,1]=np.std(np.squeeze(Singularvalues[it,isv,isr,:]))
aseries = np.linspace(0.0,1.0,nseries)
fig=plt.figure()
ax1=fig.add_subplot(111)
# for iter in range
ax1.plot(aseries,meanstdseries[0,0,:,0],'r',label=r'numerical $\tilde{\boldsymbol{X}}$')
plt.fill_between(aseries, meanstdseries[0,0,:,0]-meanstdseries[0,0,:,1], meanstdseries[0,0,:,0]+meanstdseries[0,0,:,1],
facecolor="orange", # The fill color
color='orange',       # The outline color
alpha=0.1)          # Transparency of the fill
ax1.plot(aseries,meanstdseries[1,0,:,0],'b',label=r'EI $\boldsymbol{\tilde{M},\ \tilde{N}}$')
plt.fill_between(aseries, meanstdseries[1,0,:,0]-meanstdseries[1,0,:,1], meanstdseries[1,0,:,0]+meanstdseries[1,0,:,1],
facecolor="orange", # The fill color
color='blue',       # The outline color
alpha=0.1)          # Transparency of the fill
ax1.plot(aseries,meanstdseries[2,0,:,0],'g',label=r'SVD $\boldsymbol{\tilde{U},\ \tilde{V}}$')
plt.fill_between(aseries, meanstdseries[2,0,:,0]-meanstdseries[2,0,:,1], meanstdseries[2,0,:,0]+meanstdseries[2,0,:,1],
facecolor="orange", # The fill color
color='green',       # The outline color
alpha=0.1)          # Transparency of the fill

ax1.plot(aseries,meanstdseries[0,1,:,0],'r--',label=r'numerical $\tilde{\boldsymbol{X}}$')
plt.fill_between(aseries, meanstdseries[0,1,:,0]-meanstdseries[0,1,:,1], meanstdseries[0,1,:,0]+meanstdseries[0,1,:,1],
facecolor="orange", # The fill color
color='orange',       # The outline color
alpha=0.1)          # Transparency of the fill
ax1.plot(aseries,meanstdseries[1,1,:,0],'b--',label=r'EI $\boldsymbol{\tilde{M},\ \tilde{N}}$')
plt.fill_between(aseries, meanstdseries[1,1,:,0]-meanstdseries[1,1,:,1], meanstdseries[1,1,:,0]+meanstdseries[1,1,:,1],
facecolor="orange", # The fill color
color='blue',       # The outline color
alpha=0.1)          # Transparency of the fill
ax1.plot(aseries,meanstdseries[2,1,:,0],'g--',label=r'SVD $\boldsymbol{\tilde{U},\ \tilde{V}}$')
plt.fill_between(aseries, meanstdseries[2,1,:,0]-meanstdseries[2,1,:,1], meanstdseries[2,1,:,0]+meanstdseries[2,1,:,1],
facecolor="orange", # The fill color
color='green',       # The outline color
alpha=0.1)          # Transparency of the fill
# ax1.plot(aseries,Singularvalue[1,0,:],'b',label=r'EI $\boldsymbol{\tilde{M},\ \tilde{N}}$')
# ax1.plot(aseries,Singularvalue[2,0,:],'g',label=r'SVD $\boldsymbol{\tilde{U},\ \tilde{V}}$')

# ax1.plot(aseries,Singularvalue[0,1,:],'r--',label=r'numerical $\tilde{\boldsymbol{X}}$')
# ax1.plot(aseries,Singularvalue[1,1,:],'b--',label=r'EI $\boldsymbol{\tilde{M},\ \tilde{N}}$')
# ax1.plot(aseries,Singularvalue[2,1,:],'g--',label=r'SVD $\boldsymbol{\tilde{U},\ \tilde{V}}$')

In [16]:
ax1.set_xlabel(r'$a$',fontsize=14)
ax1.set_ylabel(r'$\rho_{i}$',fontsize=14)
ax1.set_title(r'$J_E,\ J_I=$'+str(JE)+','+str(JI)+', $b=$'+str(b)+'\n $\overline{\sigma}=$'+str(grandom),fontsize=16)



Text(0.5, 1.0, '$J_E,\\ J_I=$1.0,1.45, $b=$0.5\n $\\overline{\\sigma}=$0.5')